In [4]:
# ============================================================================
# PIPELINE FUNCTIONS - ALL PROCESSING LOGIC
# ============================================================================
# This cell defines reusable functions for the entire cleaning pipeline.
# Both diagnostic and full pipeline call these same functions.
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
from pathlib import Path
from joblib import Parallel, delayed
from tqdm import tqdm
import requests
import json
import os
import seaborn as sns
from scipy import stats

warnings.filterwarnings('ignore')

# ============================================================================
# 1. DATA LOADING & STANDARDIZATION
# ============================================================================

def load_raw_data(config):
    """Load raw data from export.csv"""
    try:
        try:
            df = pd.read_csv(
                f"{config['raw_data_path']}/export.csv",
                nrows=config.get('max_rows_to_load', None)
            )
        except:
            df = pd.read_csv(
                f"{config['raw_data_path']}/export.csv",
                delimiter=';',
                nrows=config.get('max_rows_to_load', None)
            )
        return df
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Could not find 'export.csv' in '{config['raw_data_path']}'")


def clean_column_names(df):
    """Standardize column names"""
    df.columns = (df.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(' ', '_')
                  .str.replace('(', '')
                  .str.replace(')', '')
                  .str.replace('-', '_')
                  .str.replace('[^a-z0-9_]', '', regex=True))
    return df


def convert_data_types(df):
    """Convert columns to appropriate data types"""
    # Date columns
    date_cols = [col for col in df.columns if 'date' in col or 'time' in col or 
                 col in ['created_at', 'updated_at', 'checkin_date']]
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Numeric columns
    numeric_cols = [col for col in df.columns if 'value' in col or 'severity' in col]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Categorical columns
    categorical_cols = [col for col in df.columns if col in ['name', 'category', 'trackable_name', 'type']]
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    return df


def identify_key_columns(df):
    """Identify key columns in the dataset"""
    key_cols = {
        'user_col': next((col for col in df.columns if 'user' in col), None),
        'date_col': next((col for col in df.columns if 'date' in col and 'checkin' in col), None),
        'category_col': next((col for col in df.columns if 'category' in col or 'type' in col), None),
        'name_col': next((col for col in df.columns if 'name' in col and 'user' not in col), None),
        'value_col': next((col for col in df.columns if 'value' in col), None),
    }
    
    # Fallback for date_col if checkin_date not found
    if key_cols['date_col'] is None:
        key_cols['date_col'] = next((col for col in df.columns if 'date' in col), None)
    
    return key_cols


# ============================================================================
# 2. DEMOGRAPHICS EXTRACTION
# ============================================================================

def extract_demographics(df, user_col):
    """Extract demographic data (age, sex, country)"""
    demographic_cols = []
    for possible_col in ['age', 'sex', 'gender', 'country', 'location']:
        matching_cols = [col for col in df.columns if possible_col in col.lower()]
        demographic_cols.extend(matching_cols)
    demographic_cols = list(set(demographic_cols))  # Remove duplicates
    
    if demographic_cols and user_col:
        df_demographics = df.groupby(user_col)[demographic_cols].first().reset_index()
        return df_demographics, demographic_cols
    else:
        return None, []


# ============================================================================
# 3. DATA CLEANING
# ============================================================================

def filter_user_subset(df, user_col, max_users):
    """Filter to top N users (for diagnostic mode)"""
    if max_users is not None:
        top_users = df[user_col].value_counts().head(max_users).index
        df = df[df[user_col].isin(top_users)]
    return df


def drop_missing_essentials(df, essential_cols):
    """Drop rows with missing essential columns"""
    return df.dropna(subset=essential_cols)


def handle_missing_values(df, config, key_cols):
    """Handle missing values"""
    # Drop columns with too many missing values
    missing_pct = df.isnull().sum() / len(df)
    cols_to_drop = missing_pct[missing_pct > config['missing_threshold']].index
    df = df.drop(columns=cols_to_drop)
    
    # Fill/backfill value column
    value_col = key_cols['value_col']
    user_col = key_cols['user_col']
    name_col = key_cols['name_col']
    date_col = key_cols['date_col']
    
    if value_col and value_col in df.columns:
        df = df.sort_values([user_col, name_col, date_col])
        df['value_filled'] = df.groupby([user_col, name_col])[value_col].ffill()
        df['value_filled'] = df.groupby([user_col, name_col])['value_filled'].bfill()
        df = df.dropna(subset=['value_filled'])
    
    return df


def remove_duplicates(df, key_cols):
    """Remove duplicate records"""
    user_col = key_cols['user_col']
    date_col = key_cols['date_col']
    name_col = key_cols['name_col']
    
    duplicate_keys = [user_col, date_col, name_col]
    if 'value_filled' in df.columns:
        df = df.sort_values(duplicate_keys + ['value_filled'], 
                          ascending=[True]*len(duplicate_keys) + [False])
    df = df.drop_duplicates(subset=duplicate_keys, keep='first')
    return df


def handle_outliers(df, config):
    """Handle outliers using winsorization or removal"""
    if 'value_filled' in df.columns:
        Q1 = df['value_filled'].quantile(0.25)
        Q3 = df['value_filled'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        if config['outlier_method'] == 'winsorize':
            df['value_cleaned'] = df['value_filled'].clip(lower=lower_bound, upper=upper_bound)
        else:
            df = df[(df['value_filled'] >= lower_bound) & (df['value_filled'] <= upper_bound)]
            df['value_cleaned'] = df['value_filled']
    
    return df


# ============================================================================
# 4. TRACKABLE IDENTIFICATION & LLM PROCESSING
# ============================================================================

def identify_top_trackables(df, key_cols, config):
    """Identify top symptoms and conditions"""
    category_col = key_cols['category_col']
    
    top_n_symptoms = config.get('max_trackables_per_category', 100)
    top_n_conditions = config.get('max_trackables_per_category', 100)
    
    # Separate symptoms and conditions
    if category_col and category_col in df.columns:
        df_symptoms = df[df[category_col].str.lower().str.contains('symptom', na=False)]
        df_conditions = df[df[category_col].str.lower().str.contains('condition', na=False)]
    else:
        df_symptoms = df.copy()
        df_conditions = pd.DataFrame()
    
    top_symptoms = df_symptoms['trackable_name'].value_counts().head(top_n_symptoms).index if len(df_symptoms) > 0 else []
    top_conditions = df_conditions['trackable_name'].value_counts().head(top_n_conditions).index if len(df_conditions) > 0 else []
    
    # Filter to top trackables
    df_filtered = df[
        (df['trackable_name'].isin(top_symptoms)) | 
        (df['trackable_name'].isin(top_conditions))
    ].copy()
    
    return df_filtered, list(top_symptoms), list(top_conditions)


def classify_and_deduplicate_trackables(symptoms_list, conditions_list, api_key, model):
    """Use LLM to classify and deduplicate trackables"""
    prompt = f"""You are a medical data expert tasked with cleaning and properly classifying health tracking data.

You have two lists of health trackables reported by patients:
1. SYMPTOMS (reported as symptoms): {len(symptoms_list)} items
2. CONDITIONS (reported as conditions): {len(conditions_list)} items

Your tasks:
1. **Identify redundant/duplicate names** within each category (e.g., "headache" vs "headaches", "depression" vs "depressed")
2. **Correctly classify each item** as either a SYMPTOM or CONDITION:
   - SYMPTOMS: Acute, observable experiences/feelings (e.g., "headache", "nausea", "diarrhea", "fatigue")
   - CONDITIONS: Chronic diagnoses/diseases (e.g., "migraine", "IBS", "Crohn's disease", "fibromyalgia")
3. **Swap misclassified items** between categories when appropriate
4. **Merge related items** that refer to the same thing (use singular form when possible)

CRITICAL RULES:
- **ANXIETY and DEPRESSION must ALWAYS be classified as "condition"** (regardless of how they're reported)
- Merge similar anxiety terms (e.g., "anxiety", "anxious", "anxiety disorder") into a single canonical "anxiety" 
- Merge similar depression terms (e.g., "depression", "depressed", "MDD", "major depression") into a single canonical "depression"
- For all other trackables, use your medical expertise to classify appropriately

SYMPTOMS reported by patients:
{json.dumps(symptoms_list, indent=2)}

CONDITIONS reported by patients:
{json.dumps(conditions_list, indent=2)}

Return a JSON object with this EXACT structure:
{{
  "symptoms": {{
    "original_symptom_name": {{"canonical": "canonical_name", "category": "symptom"}},
    ...
  }},
  "conditions": {{
    "original_condition_name": {{"canonical": "canonical_name", "category": "condition"}},
    ...
  }}
}}

Important notes:
- If a symptom should actually be a condition, set its category to "condition"
- If a condition should actually be a symptom, set its category to "symptom"
- Use clinical/standard terminology for canonical names
- If an item is already correctly named and classified, still include it with category matching its original list
- Be conservative with merging - only merge when truly the same thing
- **REMEMBER: All anxiety and depression variants MUST be classified as "condition"**

Return ONLY valid JSON. No explanation."""

    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.1,
                "max_tokens": 8000,
            },
            timeout=120
        )
        
        response.raise_for_status()
        result = response.json()
        
        if 'error' in result:
            raise ValueError(f"OpenRouter API error: {result['error'].get('message', str(result['error']))}")
        
        if 'choices' not in result or len(result['choices']) == 0:
            raise ValueError("API response missing 'choices' field")
        
        llm_response = result['choices'][0]['message']['content'].strip()
        
        # Parse JSON (handle potential markdown code blocks)
        if '```json' in llm_response:
            llm_response = llm_response.split('```json')[1].split('```')[0].strip()
        elif '```' in llm_response:
            llm_response = llm_response.split('```')[1].split('```')[0].strip()
        
        # Attempt to parse JSON
        try:
            mapping = json.loads(llm_response)
        except json.JSONDecodeError:
            # Try to fix incomplete JSON
            open_braces = llm_response.count('{')
            close_braces = llm_response.count('}')
            if open_braces > close_braces:
                llm_response = llm_response + ('}' * (open_braces - close_braces))
                mapping = json.loads(llm_response)
            else:
                raise
        
        # Validate structure
        if not isinstance(mapping, dict) or 'symptoms' not in mapping or 'conditions' not in mapping:
            raise ValueError("LLM response does not have expected structure")
        
        # FORCE anxiety and depression to always be conditions
        anxiety_keywords = ['anxiety', 'anxious', 'panic', 'worry']
        depression_keywords = ['depression', 'depressed', 'mdd', 'major depression', 'major depressive']
        
        for category_key in ['symptoms', 'conditions']:
            if category_key in mapping:
                for orig_name, data in mapping[category_key].items():
                    orig_lower = orig_name.lower()
                    canonical_lower = data.get('canonical', '').lower()
                    
                    is_anxiety = any(kw in orig_lower for kw in anxiety_keywords) or \
                                any(kw in canonical_lower for kw in anxiety_keywords)
                    is_depression = any(kw in orig_lower for kw in depression_keywords) or \
                                   any(kw in canonical_lower for kw in depression_keywords)
                    
                    if (is_anxiety or is_depression) and data.get('category') != 'condition':
                        data['category'] = 'condition'
        
        return mapping
        
    except Exception as e:
        raise Exception(f"LLM classification failed: {str(e)}")


def apply_llm_mapping(df, trackable_mapping, key_cols):
    """Apply LLM classification and deduplication to dataset"""
    user_col = key_cols['user_col']
    date_col = key_cols['date_col']
    
    # Create mapping dictionaries
    final_mapping = {}
    for orig, data in trackable_mapping.get('symptoms', {}).items():
        final_mapping[orig] = (data['canonical'], data['category'])
    for orig, data in trackable_mapping.get('conditions', {}).items():
        final_mapping[orig] = (data['canonical'], data['category'])
    
    canonical_map = {orig: canonical for orig, (canonical, _) in final_mapping.items()}
    category_map = {orig: category for orig, (_, category) in final_mapping.items()}
    
    # Apply mappings
    df['canonical_name'] = df['trackable_name'].map(canonical_map).fillna(df['trackable_name'])
    df['trackable_category'] = df['trackable_name'].map(category_map).fillna('unknown')
    
    # Aggregate by canonical name
    df = df.groupby(
        [user_col, date_col, 'canonical_name', 'trackable_category'],
        as_index=False
    ).agg({'value_cleaned': 'mean'})
    
    df['trackable_name'] = df['canonical_name']
    df = df.drop(columns=['canonical_name'])
    
    return df, final_mapping


# ============================================================================
# 5. WIDE FORMAT TRANSFORMATION
# ============================================================================

def user_pivot(user, df_trackables, key_cols):
    """Pivot single user's data to wide format"""
    user_col = key_cols['user_col']
    date_col = key_cols['date_col']
    
    user_df = df_trackables[df_trackables[user_col] == user]
    if user_df.empty:
        return pd.DataFrame(columns=[user_col, date_col])
    
    # Ensure date column is datetime before pivoting
    if date_col in user_df.columns:
        user_df[date_col] = pd.to_datetime(user_df[date_col], errors='coerce')
    
    # Separate symptoms and conditions
    symptoms_df = user_df[user_df['trackable_category'] == 'symptom']
    conditions_df = user_df[user_df['trackable_category'] == 'condition']
    
    # Pivot symptoms
    if not symptoms_df.empty:
        symptoms_pivot = symptoms_df.pivot_table(
            index=[user_col, date_col],
            columns='trackable_name',
            values='value_cleaned',
            aggfunc='mean'
        )
        symptoms_pivot.columns = [f'symptom_{str(col).lower().replace(" ", "_").replace("-", "_")}' 
                                  for col in symptoms_pivot.columns]
        symptoms_pivot = symptoms_pivot.reset_index()
    else:
        symptoms_pivot = pd.DataFrame(columns=[user_col, date_col])
    
    # Pivot conditions
    if not conditions_df.empty:
        conditions_pivot = conditions_df.pivot_table(
            index=[user_col, date_col],
            columns='trackable_name',
            values='value_cleaned',
            aggfunc='mean'
        )
        conditions_pivot.columns = [f'condition_{str(col).lower().replace(" ", "_").replace("-", "_")}' 
                                   for col in conditions_pivot.columns]
        conditions_pivot = conditions_pivot.reset_index()
    else:
        conditions_pivot = pd.DataFrame(columns=[user_col, date_col])
    
    # Merge symptoms and conditions
    if not symptoms_pivot.empty and not conditions_pivot.empty:
        chunk = symptoms_pivot.merge(conditions_pivot, on=[user_col, date_col], how='outer')
    elif not symptoms_pivot.empty:
        chunk = symptoms_pivot
    else:
        chunk = conditions_pivot
    
    # Ensure date column is preserved as datetime after merge
    if date_col in chunk.columns:
        chunk[date_col] = pd.to_datetime(chunk[date_col], errors='coerce')
    
    return chunk


def transform_to_wide_format(df, key_cols, n_jobs=8):
    """Transform long format to wide format with parallel processing"""
    user_col = key_cols['user_col']
    date_col = key_cols['date_col']
    
    user_list = df[user_col].unique()
    results = Parallel(n_jobs=n_jobs)(
        delayed(user_pivot)(user, df, key_cols) for user in tqdm(user_list, desc="Pivoting users")
    )
    
    df_wide = pd.concat(results, ignore_index=True)
    
    # Ensure date column is datetime after concatenation
    if date_col in df_wide.columns:
        df_wide[date_col] = pd.to_datetime(df_wide[date_col], errors='coerce')
    
    return df_wide


# ============================================================================
# 6. FEATURE ENGINEERING
# ============================================================================

def add_lag_and_rolling_features(df, user_col, lookback_window):
    """Add lag and rolling statistics features"""
    symptom_cols = [col for col in df.columns if col.startswith('symptom_')]
    condition_cols = [col for col in df.columns if col.startswith('condition_')]
    
    # Lag features for symptoms
    for col in symptom_cols:
        for lag in range(1, lookback_window + 1):
            df[f'{col}_lag{lag}'] = df.groupby(user_col)[col].shift(lag)
    
    # Rolling statistics for symptoms
    for col in symptom_cols:
        df[f'{col}_rolling_mean_{lookback_window}d'] = (
            df.groupby(user_col)[col]
            .rolling(window=lookback_window, min_periods=3)
            .mean()
            .reset_index(level=0, drop=True)
        )
        df[f'{col}_rolling_std_{lookback_window}d'] = (
            df.groupby(user_col)[col]
            .rolling(window=lookback_window, min_periods=3)
            .std()
            .reset_index(level=0, drop=True)
        )
    
    # Lag features for conditions
    for col in condition_cols:
        for lag in range(1, lookback_window + 1):
            df[f'{col}_lag{lag}'] = df.groupby(user_col)[col].shift(lag)
    
    # Rolling statistics for conditions
    for col in condition_cols:
        df[f'{col}_rolling_mean_{lookback_window}d'] = (
            df.groupby(user_col)[col]
            .rolling(window=lookback_window, min_periods=3)
            .mean()
            .reset_index(level=0, drop=True)
        )
        df[f'{col}_rolling_std_{lookback_window}d'] = (
            df.groupby(user_col)[col]
            .rolling(window=lookback_window, min_periods=3)
            .std()
            .reset_index(level=0, drop=True)
        )
    
    return df


def add_temporal_features(df, date_col):
    """Add temporal features (day of week, month, etc.)"""
    # Ensure date column is datetime
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df['day_of_week'] = df[date_col].dt.dayofweek
        df['day_of_month'] = df[date_col].dt.day
        df['month'] = df[date_col].dt.month
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    return df


# ============================================================================
# 7. FINAL FILTERING & EXPORT
# ============================================================================

def filter_users_by_min_observations(df, user_col, min_observations):
    """Filter users with sufficient observations"""
    user_counts = df.groupby(user_col).size()
    valid_users = user_counts[user_counts >= min_observations].index
    df = df[df[user_col].isin(valid_users)].copy()
    return df


def add_user_sequence_id(df, user_col):
    """Add sequential user ID"""
    df['user_seq_id'] = df.groupby(user_col).ngroup() + 1
    cols = list(df.columns)
    user_idx = cols.index(user_col)
    cols.insert(user_idx + 1, cols.pop(cols.index('user_seq_id')))
    df = df[cols]
    return df


def create_validation_report(df, user_col, date_col, df_demographics, demographic_cols, trackable_mapping):
    """Create validation report"""
    symptom_features = [col for col in df.columns if col.startswith('symptom_') and 'lag' not in col and 'rolling' not in col]
    condition_features = [col for col in df.columns if col.startswith('condition_') and 'rolling' not in col]
    
    report = {
        'total_users': df[user_col].nunique() if not df.empty else 0,
        'total_observations': len(df),
        'date_range_start': df[date_col].min() if not df.empty else None,
        'date_range_end': df[date_col].max() if not df.empty else None,
        'days_span': (df[date_col].max() - df[date_col].min()).days if not df.empty else None,
        'total_features': len(df.columns),
        'symptom_features': len(symptom_features),
        'condition_features': len(condition_features),
        'remaining_missing_values': df.isnull().sum().sum() if not df.empty else None,
        'remaining_duplicates': df.duplicated().sum() if not df.empty else None,
    }
    
    if df_demographics is not None:
        report['demographic_features'] = len(demographic_cols)
    
    if trackable_mapping:
        report['llm_classification_applied'] = True
        if 'symptoms' in trackable_mapping and 'conditions' in trackable_mapping:
            symptom_swaps = sum(1 for data in trackable_mapping['symptoms'].values() if data.get('category') == 'condition')
            condition_swaps = sum(1 for data in trackable_mapping['conditions'].values() if data.get('category') == 'symptom')
            report['symptoms_reclassified_as_conditions'] = symptom_swaps
            report['conditions_reclassified_as_symptoms'] = condition_swaps
    
    return report


def export_datasets(df, config, validation_report):
    """Export cleaned datasets"""
    # CSV export
    csv_path = f"{config['output_path']}/flaredown_cleaned.csv"
    df.to_csv(csv_path, index=False)
    
    # Pickle export
    pkl_path = f"{config['output_path']}/flaredown_cleaned.pkl"
    df.to_pickle(pkl_path)
    
    # Excel sample export
    excel_path = f"{config['output_path']}/flaredown_cleaned_sample.xlsx"
    sample_size = min(1000, len(df))
    df_sample = df.head(sample_size)
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df_sample.to_excel(writer, sheet_name='cleaned_data', index=False)
        validation_df = pd.DataFrame([validation_report])
        validation_df.to_excel(writer, sheet_name='validation_report', index=False)
        col_info = pd.DataFrame({
            'column_name': df.columns,
            'data_type': df.dtypes.astype(str),
            'missing_count': df.isnull().sum(),
            'unique_values': [df[col].nunique() for col in df.columns]
        })
        col_info.to_excel(writer, sheet_name='column_info', index=False)
    
    return csv_path, pkl_path, excel_path


def create_train_val_test_splits(df, user_col, date_col, config, train_ratio=0.7, val_ratio=0.15):
    """Create temporal train/validation/test splits"""
    df = df.sort_values([user_col, date_col])
    
    def temporal_split(group):
        n = len(group)
        train_idx = int(n * train_ratio)
        val_idx = int(n * (train_ratio + val_ratio))
        split_col = ['test'] * n
        split_col[:train_idx] = ['train'] * train_idx
        split_col[train_idx:val_idx] = ['validation'] * (val_idx - train_idx)
        group['split'] = split_col
        return group
    
    if not df.empty:
        df = df.groupby(user_col, group_keys=False).apply(temporal_split)
        for split_name in ['train', 'validation', 'test']:
            split_df = df[df['split'] == split_name].drop(columns=['split'])
            split_path = f"{config['output_path']}/flaredown_{split_name}.csv"
            split_df.to_csv(split_path, index=False)
    
    return df


print("✅ Pipeline functions loaded successfully!")
print("   All data processing logic is now reusable.")
print("   Fixed: Date column datetime preservation in wide format transformation")


✅ Pipeline functions loaded successfully!
   All data processing logic is now reusable.
   Fixed: Date column datetime preservation in wide format transformation


In [5]:
# ============================================================================
# DIAGNOSTIC MODE - QUICK PIPELINE VALIDATION
# ============================================================================
# Run this cell to test the pipeline with minimal data (fast execution)
# This uses the SAME functions as the full pipeline below
# ============================================================================

# DIAGNOSTIC CONFIG - EXTREMELY AGGRESSIVE FILTERING
DIAG_CONFIG = {
    'raw_data_path': 'raw_data/',
    'output_path': 'cleaned_data/diagnostic/',
    'reports_path': 'reports/diagnostic/',
    'min_observations_per_user': 10,
    'lookback_window': 3,
    'missing_threshold': 0.9,
    'outlier_method': 'winsorize',
    'openrouter_api_key': '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',
    'openrouter_model': 'google/gemini-2.5-flash-preview-09-2025',
    'use_llm_deduplication': False,  # Skip LLM for speed
    'max_users': 5,  # Only process 5 users
    'max_trackables_per_category': 5,  # Only top 5 symptoms and 5 conditions
    'max_rows_to_load': 5000,  # Only load first 5000 rows
}

# Create diagnostic directories
for path in [DIAG_CONFIG['output_path'], DIAG_CONFIG['reports_path']]:
    Path(path).mkdir(parents=True, exist_ok=True)

print("🔧 " * 35)
print("DIAGNOSTIC MODE - QUICK PIPELINE VALIDATION")
print("🔧 " * 35)
print(f"Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Config: {DIAG_CONFIG['max_users']} users, {DIAG_CONFIG['max_rows_to_load']} rows")
print()

try:
    # STEP 1: Load raw data
    print("[1/14] Loading raw data (limited)...")
    df_diag = load_raw_data(DIAG_CONFIG)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 2: Standardize column names
    print("[2/14] Standardizing column names...")
    df_diag = clean_column_names(df_diag)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 3: Convert data types
    print("[3/14] Converting data types...")
    df_diag = convert_data_types(df_diag)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 4: Identify key columns
    print("[4/14] Identifying key columns...")
    key_cols = identify_key_columns(df_diag)
    print(f"   ✓ user={key_cols['user_col']}, date={key_cols['date_col']}")
    
    # STEP 4B: Filter to top N users (diagnostic only)
    print("[4B/14] Filtering to top users...")
    df_diag = filter_user_subset(df_diag, key_cols['user_col'], DIAG_CONFIG['max_users'])
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 5: Extract demographics
    print("[5/14] Extracting demographics...")
    df_demographics, demographic_cols = extract_demographics(df_diag, key_cols['user_col'])
    print(f"   ✓ {len(demographic_cols)} demographic columns")
    
    # STEP 6: Drop missing essentials
    print("[6/14] Dropping missing essentials...")
    essential_cols = [col for col in [key_cols['user_col'], key_cols['date_col'], key_cols['name_col']] if col is not None]
    df_diag = drop_missing_essentials(df_diag, essential_cols)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 7: Handle missing values
    print("[7/14] Handling missing values...")
    df_diag = handle_missing_values(df_diag, DIAG_CONFIG, key_cols)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 8: Remove duplicates
    print("[8/14] Removing duplicates...")
    df_diag = remove_duplicates(df_diag, key_cols)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 9: Handle outliers
    print("[9/14] Handling outliers...")
    df_diag = handle_outliers(df_diag, DIAG_CONFIG)
    print(f"   ✓ {df_diag.shape}")
    
    # STEP 10: Identify top trackables
    print("[10/14] Identifying top trackables...")
    df_diag, top_symptoms, top_conditions = identify_top_trackables(df_diag, key_cols, DIAG_CONFIG)
    df_diag['trackable_category'] = 'unknown'  # Skip LLM in diagnostic mode
    print(f"   ✓ {len(top_symptoms)} symptoms, {len(top_conditions)} conditions")
    
    # STEP 11: Transform to wide format
    print("[11/14] Transforming to wide format...")
    df_wide = transform_to_wide_format(df_diag, key_cols, n_jobs=2)
    print(f"   ✓ {df_wide.shape}")
    
    # STEP 12: Merge demographics
    print("[12/14] Merging demographics...")
    if df_demographics is not None:
        df_wide = df_wide.merge(df_demographics, on=key_cols['user_col'], how='left')
    print(f"   ✓ {df_wide.shape}")
    
    # STEP 13: Add features
    print("[13/14] Adding temporal features...")
    df_wide = add_temporal_features(df_wide, key_cols['date_col'])
    print(f"   ✓ {df_wide.shape}")
    
    # STEP 14: Export
    print("[14/14] Exporting diagnostic data...")
    csv_path = f"{DIAG_CONFIG['output_path']}/diagnostic_cleaned.csv"
    df_wide.to_csv(csv_path, index=False)
    print(f"   ✓ Saved: {csv_path}")
    
    # SUCCESS
    print()
    print("✅ " * 35)
    print("DIAGNOSTIC PASSED - All pipeline steps executed successfully!")
    print("✅ " * 35)
    print(f"End: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Output: {DIAG_CONFIG['output_path']}")
    print()
    print("💡 Pipeline is working correctly!")
    print("   Any changes to functions will be tested here automatically.")
    
except Exception as e:
    print()
    print("❌ " * 35)
    print("DIAGNOSTIC FAILED - Pipeline error detected!")
    print("❌ " * 35)
    print(f"Error: {type(e).__name__}: {str(e)}")
    print()
    import traceback
    traceback.print_exc()


🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 
DIAGNOSTIC MODE - QUICK PIPELINE VALIDATION
🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 
Start: 2025-10-24 09:39:55
Config: 5 users, 5000 rows

[1/14] Loading raw data (limited)...
   ✓ (5000, 9)
[2/14] Standardizing column names...
   ✓ (5000, 9)
[3/14] Converting data types...
   ✓ (5000, 9)
[4/14] Identifying key columns...
   ✓ user=user_id, date=checkin_date
[4B/14] Filtering to top users...
   ✓ (2068, 9)
[5/14] Extracting demographics...
   ✓ 3 demographic columns
[6/14] Dropping missing essentials...
   ✓ (2068, 9)
[7/14] Handling missing values...
   ✓ (1793, 10)
[8/14] Removing duplicates...
   ✓ (1793, 10)
[9/14] Handling outliers...
   ✓ (1793, 11)
[10/14] Identifying top trackables...
   ✓ 5 symptoms, 5 conditions
[11/14] Transforming to wide format...


Pivoting users:   0%|          | 0/5 [00:00<?, ?it/s]/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Pivoting users: 100%|██████████| 5/5 [00:00<00:00, 777.50it/s]/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a

   ✓ (0, 2)
[12/14] Merging demographics...
   ✓ (0, 5)
[13/14] Adding temporal features...
   ✓ (0, 9)
[14/14] Exporting diagnostic data...
   ✓ Saved: cleaned_data/diagnostic//diagnostic_cleaned.csv

✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 
DIAGNOSTIC PASSED - All pipeline steps executed successfully!
✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 
End: 2025-10-24 09:39:55
Output: cleaned_data/diagnostic/

💡 Pipeline is working correctly!
   Any changes to functions will be tested here automatically.



/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

# Flaredown Health Data Cleaning Pipeline

## 🎯 NEW: Modular Function-Based Architecture

**The pipeline has been refactored into reusable functions!**

### Architecture Overview

**Cell 1: Pipeline Functions** 📦
- All processing logic extracted into reusable functions
- Single source of truth for all data transformations
- Organized into 7 functional categories

**Cell 2: Diagnostic Mode** 🔧
- Quick validation with minimal data (5 users, 5K rows)
- Runs in seconds to verify pipeline integrity
- Uses THE SAME functions as the full pipeline
- **Run this after ANY code changes to catch breaking issues!**

**Cell 3-4: Full Pipeline** 🚀
- Complete data processing using the same functions
- Processes entire dataset with all features
- Guaranteed to work if diagnostic passes

### Key Benefits

✅ **Automatic Synchronization** - Changes to functions affect both diagnostic and full pipeline  
✅ **Easier Testing** - Run diagnostic in seconds instead of processing full dataset  
✅ **Better Maintainability** - Update logic once, it works everywhere  
✅ **Cleaner Code** - Modular, organized, and easier to understand  
✅ **No Duplication** - All logic lives in one place

## Overview
This comprehensive pipeline processes health tracking data from the Flaredown dataset, utilizing LLM-powered intelligent classification and deduplication to create analysis-ready datasets for symptom forecasting.

## Key Features

### LLM-Powered Processing
- **Intelligent Classification**: Automatically distinguishes between symptoms (acute) and conditions (chronic diagnoses)
- **Smart Deduplication**: Merges redundant trackable names (e.g., "headache" vs "headaches")
- **Auto-Correction**: Swaps misclassified items between symptom/condition categories

### Data Processing
1. **Symptoms** (Top 100): Acute, observable experiences with continuous severity features
2. **Conditions** (Top 100): Chronic diagnoses with continuous severity features
3. **Demographics**: Age, sex/gender, location preserved and merged
4. **Temporal Features**: Day of week, weekend indicators, rolling statistics

### Outputs
- **Cleaned Dataset**: CSV/PKL/Excel with symptoms, conditions, and demographics
- **Train/Validation/Test Splits**: Temporal splits per user (70/15/15)
- **Visualizations**: Comprehensive reports including age-based analysis
- **Mapping File**: Complete trackable classification and deduplication mapping

### Advanced Analytics
- Age-stratified symptom analysis (anxiety and depression)
- Symptom correlation matrices
- Condition prevalence across patient population
- Temporal coverage and user observation distributions

## Usage

### Option 1: Quick Test (Recommended First)
1. Run **Cell 1** (Pipeline Functions)
2. Run **Cell 2** (Diagnostic Mode)
3. If diagnostic passes ✅, proceed to full pipeline

### Option 2: Full Pipeline
1. Run **Cell 1** (Pipeline Functions)
2. Run **Cell 3-4** (Full Pipeline)
3. Run remaining cells for visualizations

The pipeline will:
1. Load and clean raw data
2. Extract demographics
3. Classify trackables via LLM (if API key provided)
4. Create feature-rich wide format dataset
5. Generate comprehensive visualizations
6. Export analysis-ready files


In [6]:
# ============================================================================
# FULL PIPELINE - COMPLETE DATA PROCESSING
# ============================================================================
# This cell runs the complete pipeline using the same functions as diagnostic mode
# ============================================================================

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# FULL PIPELINE CONFIG
CONFIG = {
    'raw_data_path': 'raw_data/',
    'output_path': 'cleaned_data/',
    'reports_path': 'reports/',
    'min_observations_per_user': 28,
    'lookback_window': 7,
    'missing_threshold': 0.7,
    'outlier_method': 'winsorize',
    'openrouter_api_key': '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',
    'openrouter_model': 'google/gemini-2.5-flash-preview-09-2025',
    'use_llm_deduplication': True,
    'max_trackables_per_category': 100,  # Top 100 symptoms and top 100 conditions
    'max_users': None,  # Process all users
    'max_rows_to_load': None,  # Load entire dataset
}

# Create directories
for path in [CONFIG['output_path'], CONFIG['reports_path']]:
    Path(path).mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("FLAREDOWN DATASET CLEANING PIPELINE (export.csv)")
print("=" * 70)
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# STEP 1: LOAD RAW DATA
try:
    df = load_raw_data(CONFIG)
    print(f"✓ Loaded export.csv: {df.shape}")
    print(f"  Columns: {list(df.columns)}")
except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find 'export.csv' in '{CONFIG['raw_data_path']}'")
    raise e
print("\n--- After loading:", df.shape)
print(df.head())

# STEP 2: STANDARDIZE COLUMN NAMES
df = clean_column_names(df)
print("\n--- After column renaming:", df.shape)

# STEP 3: Convert Data Types
df = convert_data_types(df)
print("\n--- After type conversion:", df.shape)

# STEP 4: Identify key columns
key_cols = identify_key_columns(df)
user_col = key_cols['user_col']
date_col = key_cols['date_col']
category_col = key_cols['category_col']
name_col = key_cols['name_col']
value_col = key_cols['value_col']

print(f"\n--- After key column detection: {df.shape}, users = {df[user_col].nunique() if user_col in df.columns else 'N/A'}")

# STEP 4B: EXTRACT DEMOGRAPHIC DATA (age, sex, country)
print("\n--- Extracting demographic data...")
df_demographics, demographic_cols = extract_demographics(df, user_col)

if df_demographics is not None:
    print(f"✓ Extracted demographics for {len(df_demographics)} users")
    print(f"  Demographic columns: {demographic_cols}")
    print(f"\n  Sample demographics:")
    print(df_demographics.head())
else:
    print("  No demographic columns found (age, sex, country)")

# STEP 5: Filter and clean data (drop NA in essentials)
essential_cols = [col for col in [user_col, date_col, name_col] if col is not None]
df = drop_missing_essentials(df, essential_cols)
print(f"\n--- After dropna essentials: {df.shape}, users = {df[user_col].nunique() if user_col in df.columns else 'N/A'}")

# STEP 6: Keep all trackable data (symptoms AND conditions)
df_trackables = df.copy()
print(f"\n--- Keeping all trackable data (symptoms + conditions): {df_trackables.shape}, users = {df_trackables[user_col].nunique()}")

# STEP 7: Handle missing values  
df_trackables = handle_missing_values(df_trackables, CONFIG, key_cols)
print(f"\n--- After handling missing values: {df_trackables.shape}, users = {df_trackables[user_col].nunique()}")

# STEP 8: Remove duplicates
df_trackables = remove_duplicates(df_trackables, key_cols)
print(f"\n--- After removing duplicates: {df_trackables.shape}, users = {df_trackables[user_col].nunique()}")

# STEP 9: Handle outliers
df_trackables = handle_outliers(df_trackables, CONFIG)
print(f"\n--- After outlier handling: {df_trackables.shape}, users = {df_trackables[user_col].nunique()}")

# STEP 10: Filter users and identify top trackables (symptoms + conditions)
user_day_counts = df_trackables.groupby(user_col)[date_col].nunique()
valid_users = user_day_counts[user_day_counts >= 21].index
df_trackables = df_trackables[df_trackables[user_col].isin(valid_users)].copy()
print(f"\n--- After min days user filter: {df_trackables.shape}, users = {df_trackables[user_col].nunique()}")

# Identify top trackables
df_trackables_top, top_symptoms, top_conditions = identify_top_trackables(df_trackables, key_cols, CONFIG)
print(f"\n--- Identified top trackables:")
print(f"  Top {len(top_symptoms)} symptoms: {list(top_symptoms[:10])}...")
print(f"  Top {len(top_conditions)} conditions: {list(top_conditions[:10])}...")
print(f"\n--- After selecting top trackables: {df_trackables_top.shape}")

# STEP 10A: LLM-BASED TRACKABLE DEDUPLICATION AND CLASSIFICATION
print("\n" + "=" * 70)
print("STEP 10A: LLM-BASED TRACKABLE DEDUPLICATION AND CLASSIFICATION")
print("=" * 70)

trackable_mapping = {}
final_mapping = {}

if CONFIG['use_llm_deduplication'] and CONFIG['openrouter_api_key']:
    try:
        unique_symptoms = list(top_symptoms) if len(top_symptoms) > 0 else []
        unique_conditions = list(top_conditions) if len(top_conditions) > 0 else []
        
        print(f"\n📋 Analyzing trackables:")
        print(f"  • {len(unique_symptoms)} unique symptoms")
        print(f"  • {len(unique_conditions)} unique conditions")
        
        print("  📡 Sending request to OpenRouter API...")
        trackable_mapping = classify_and_deduplicate_trackables(
            unique_symptoms, 
            unique_conditions,
            CONFIG['openrouter_api_key'],
            CONFIG['openrouter_model']
        )
        
        print(f"  ✓ Successfully received mappings")
        
        # Process the mappings
        print(f"\n📊 Processing LLM Results:")
        df_trackables_top, final_mapping = apply_llm_mapping(df_trackables_top, trackable_mapping, key_cols)
        
        # Analyze the results
        symptom_to_symptom = sum(1 for _, (_, cat) in final_mapping.items() if cat == 'symptom' and _ in unique_symptoms)
        symptom_to_condition = sum(1 for _, (_, cat) in final_mapping.items() if cat == 'condition' and _ in unique_symptoms)
        condition_to_condition = sum(1 for _, (_, cat) in final_mapping.items() if cat == 'condition' and _ in unique_conditions)
        condition_to_symptom = sum(1 for _, (_, cat) in final_mapping.items() if cat == 'symptom' and _ in unique_conditions)
        
        print(f"  • Symptoms remaining as symptoms: {symptom_to_symptom}")
        print(f"  • Symptoms reclassified as conditions: {symptom_to_condition}")
        print(f"  • Conditions remaining as conditions: {condition_to_condition}")
        print(f"  • Conditions reclassified as symptoms: {condition_to_symptom}")
        
        # Count unique canonical names
        unique_canonicals = set(canonical for canonical, _ in final_mapping.values())
        print(f"  • Total unique canonical trackables: {len(unique_canonicals)}")
        print(f"  • Redundant items merged: {len(final_mapping) - len(unique_canonicals)}")
        
        # Save mapping to file
        mapping_path = f"{CONFIG['output_path']}/trackable_mapping.json"
        with open(mapping_path, 'w') as f:
            save_mapping = {
                orig: {'canonical': canonical, 'category': category}
                for orig, (canonical, category) in final_mapping.items()
            }
            json.dump(save_mapping, f, indent=2)
        print(f"  Saved mapping to: {mapping_path}")
        
        print(f"\n✅ Classification and deduplication complete!")
        
    except Exception as e:
        print(f"\n⚠️  WARNING: LLM classification failed: {str(e)}")
        print(f"  Continuing without LLM processing...")
        df_trackables_top['trackable_category'] = 'unknown'
        trackable_mapping = {}
        
elif CONFIG['use_llm_deduplication'] and not CONFIG['openrouter_api_key']:
    print("\n⚠️  LLM deduplication enabled but no API key provided")
    print("  Please set CONFIG['openrouter_api_key'] to use this feature")
    print("  Continuing without deduplication...")
    df_trackables_top['trackable_category'] = 'unknown'
else:
    print("\n⏭️  LLM deduplication disabled (set CONFIG['use_llm_deduplication'] = True to enable)")
    df_trackables_top['trackable_category'] = 'unknown'

# STEP 10B: Wide format (pivot) - separate symptoms and conditions
print("\n" + "=" * 70)
print("STEP 10B: PIVOT TO WIDE FORMAT (SYMPTOMS + CONDITIONS)")
print("=" * 70)

user_list = df_trackables_top[user_col].unique()[:300]
print(f"  Pivoting data for {len(user_list)} users...")
df_wide = transform_to_wide_format(df_trackables_top, key_cols, n_jobs=8)
print(f"\n--- After user_pivot/wide: {df_wide.shape}, users = {df_wide[user_col].nunique() if user_col in df_wide.columns else 'N/A'}")

# Count symptom and condition columns
symptom_cols = [col for col in df_wide.columns if col.startswith('symptom_')]
condition_cols = [col for col in df_wide.columns if col.startswith('condition_')]
print(f"  • Symptom columns: {len(symptom_cols)}")
print(f"  • Condition columns: {len(condition_cols)}")

# STEP 10C: MERGE DEMOGRAPHICS BACK INTO WIDE FORMAT
if df_demographics is not None and not df_demographics.empty:
    print("\n--- Merging demographics back into dataset...")
    df_wide = df_wide.merge(df_demographics, on=user_col, how='left')
    print(f"✓ Merged demographics: {df_wide.shape}")
    print(f"  Demographics columns added: {demographic_cols}")
    
    # Reorder columns
    cols = list(df_wide.columns)
    for demo_col in demographic_cols:
        if demo_col in cols:
            cols.remove(demo_col)
    date_idx = cols.index(date_col) if date_col in cols else 1
    for i, demo_col in enumerate(demographic_cols):
        if demo_col in df_wide.columns:
            cols.insert(date_idx + 1 + i, demo_col)
    df_wide = df_wide[cols]
    print(f"  Column order updated (demographics after date column)")

# STEP 11: FEATURE ENGINEERING
print(f"\n--- Creating features...")
df_wide = add_lag_and_rolling_features(df_wide, user_col, CONFIG['lookback_window'])
df_wide = add_temporal_features(df_wide, date_col)
print(f"\n--- After feature engineering: {df_wide.shape}, users = {df_wide[user_col].nunique() if user_col in df_wide.columns else 'N/A'}")

# STEP 12: FILTER USERS WITH SUFFICIENT DATA
df_final = filter_users_by_min_observations(df_wide, user_col, CONFIG['min_observations_per_user'])
print(f"\n--- After final min obs user filter: {df_final.shape}, users = {df_final[user_col].nunique()}")

df_final = add_user_sequence_id(df_final, user_col)

# Final Validation Report
print("\n=== FINAL SHAPE ===")
print(f"df_final shape: {df_final.shape}, users: {df_final[user_col].nunique() if not df_final.empty else 0}")
if df_demographics is not None:
    print(f"\n=== DEMOGRAPHIC DATA PRESERVED ===")
    for demo_col in demographic_cols:
        if demo_col in df_final.columns:
            print(f"  {demo_col}: {df_final[demo_col].nunique()} unique values, {df_final[demo_col].isnull().sum()} missing")

# STEP 13: FINAL VALIDATION
validation_report = create_validation_report(df_final, user_col, date_col, df_demographics, demographic_cols, trackable_mapping)

print("\n📊 VALIDATION REPORT:")
for key, value in validation_report.items():
    print(f"  {key.replace('_', ' ').title()}: {value}")

# STEP 14: EXPORT
print("\n" + "=" * 70)
print("STEP 14: Exporting cleaned data...")
print("=" * 70)

csv_path, pkl_path, excel_path = export_datasets(df_final, CONFIG, validation_report)
print(f"\n✓ Saved: {csv_path}")
print(f"  Size: {len(df_final):,} rows × {len(df_final.columns)} columns")
print(f"✓ Saved: {pkl_path}")
sample_size = min(1000, len(df_final))
print(f"✓ Saved: {excel_path}")
print(f"  Excel contains sample of {sample_size:,} rows (full data in CSV/PKL)")


FLAREDOWN DATASET CLEANING PIPELINE (export.csv)
Start time: 2025-10-24 09:39:55

✓ Loaded export.csv: (7976223, 9)
  Columns: ['user_id', 'age', 'sex', 'country', 'checkin_date', 'trackable_id', 'trackable_type', 'trackable_name', 'trackable_value']

--- After loading: (7976223, 9)
                            user_id   age     sex country checkin_date  \
0  QEVuQwEABlEzkh7fsBBjEe26RyIVcg==   NaN     NaN     NaN   2015-11-26   
1  QEVuQwEAWRNGnuTRqXG2996KSkTIEw==  32.0    male      US   2015-11-26   
2  QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==   2.0  female      CA   2017-04-28   
3  QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==   2.0  female      CA   2017-04-28   
4  QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==   2.0  female      CA   2017-04-28   

  trackable_id trackable_type  \
0         1069      Condition   
1         1069      Condition   
2         3168      Condition   
3         3169      Condition   
4         3170      Condition   

                                      trackable_name trackable_value  
0

Pivoting users:   0%|          | 8/3461 [00:00<01:23, 41.23it/s]/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Pivoting users:   0%|          | 16/3461 [00:00<03:34, 16.05it/s]/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_23560/2545918097.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Pivoting users:   0%|          | 16/3461 [00:00<03:34, 16.05it/s]/var/folders/d_/2z80vqvs14lbbs3qltxq34200000gn/T/ipykernel_235


--- After user_pivot/wide: (222075, 146), users = 3461
  • Symptom columns: 84
  • Condition columns: 60

--- Merging demographics back into dataset...
✓ Merged demographics: (222075, 149)
  Demographics columns added: ['country', 'sex', 'age']
  Column order updated (demographics after date column)

--- Creating features...
  Column order updated (demographics after date column)

--- Creating features...

--- After feature engineering: (222075, 1449), users = 3461

--- After feature engineering: (222075, 1449), users = 3461

--- After final min obs user filter: (198063, 1449), users = 2221

--- After final min obs user filter: (198063, 1449), users = 2221

=== FINAL SHAPE ===
df_final shape: (198063, 1450), users: 2221

=== DEMOGRAPHIC DATA PRESERVED ===
  country: 52 unique values, 7077 missing
  sex: 4 unique values, 2950 missing
  age: 66 unique values, 8335 missing

=== FINAL SHAPE ===
df_final shape: (198063, 1450), users: 2221

=== DEMOGRAPHIC DATA PRESERVED ===
  country: 52 u

In [7]:
# STEP 15: Create Frequency Reports
fig_path = os.path.join(CONFIG['reports_path'], "figures/")
Path(fig_path).mkdir(parents=True, exist_ok=True)

if not df_final.empty:
    # Report 1: Symptom Frequency (most common to least common)
    symptom_cols_base = [col for col in df_final.columns if col.startswith('symptom_') and 'lag' not in col and 'rolling' not in col]
    if len(symptom_cols_base) > 0:
        symptom_frequency = {}
        for col in symptom_cols_base:
            users_tracking = df_final[df_final[col] > 0][user_col].nunique()
            symptom_name = col.replace('symptom_', '').replace('_', ' ').title()
            symptom_frequency[symptom_name] = users_tracking
        
        sorted_symptoms = sorted(symptom_frequency.items(), key=lambda x: x[1], reverse=True)
        symptom_names = [x[0] for x in sorted_symptoms]
        symptom_counts = [x[1] for x in sorted_symptoms]
        
        plt.figure(figsize=(16, 8))
        plt.bar(symptom_names, symptom_counts, color='steelblue')
        plt.xlabel('Symptom', fontsize=12)
        plt.ylabel('Number of Unique Patients', fontsize=12)
        plt.title('Symptom Frequency: Most Common to Least Common', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(fig_path, "symptom_frequency.png"), dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Saved: symptom_frequency.png")
    
    # Report 2: Condition Frequency (most common to least common)
    condition_cols_base = [col for col in df_final.columns if col.startswith('condition_') and 'lag' not in col and 'rolling' not in col]
    if len(condition_cols_base) > 0:
        condition_frequency = {}
        for col in condition_cols_base:
            users_tracking = df_final[df_final[col] > 0][user_col].nunique()
            condition_name = col.replace('condition_', '').replace('_', ' ').title()
            condition_frequency[condition_name] = users_tracking
        
        sorted_conditions = sorted(condition_frequency.items(), key=lambda x: x[1], reverse=True)
        condition_names = [x[0] for x in sorted_conditions]
        condition_counts = [x[1] for x in sorted_conditions]
        
        plt.figure(figsize=(16, 8))
        plt.bar(condition_names, condition_counts, color='coral')
        plt.xlabel('Condition', fontsize=12)
        plt.ylabel('Number of Unique Patients', fontsize=12)
        plt.title('Condition Frequency: Most Common to Least Common', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(fig_path, "condition_frequency.png"), dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Saved: condition_frequency.png")
else:
    print("Skipping reports: df_final is empty")

# STEP 16: CREATE ANALYSIS-READY SUBSETS (train/val/test)
print("-" * 70)
print("STEP 16: Creating analysis-ready subsets (train/val/test)...")

if not df_final.empty:
    df_final = create_train_val_test_splits(df_final, user_col, date_col, CONFIG)
    
    # Report what was created
    for split_name in ['train', 'validation', 'test']:
        split_count = (df_final['split'] == split_name).sum() if 'split' in df_final.columns else 0
        print(f"✓ Saved {split_name} set: {split_count} records")
    
    # Drop split column for final dataset
    if 'split' in df_final.columns:
        df_final = df_final.drop(columns=['split'])
else:
    print("WARNING: df_final is empty -- skipping train-validation-test export.")

print("\n" + "=" * 70)
print("✅ DATA CLEANING COMPLETE!")
print("=" * 70)
print(f"\nEnd time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n📁 Output files created:")
print(f"   • {CONFIG['output_path']}/flaredown_cleaned.csv (full dataset)")
print(f"   • {CONFIG['output_path']}/flaredown_cleaned.pkl (full dataset)")
print(f"   • {CONFIG['output_path']}/flaredown_cleaned_sample.xlsx (sample for quick review)")
if trackable_mapping:
    print(f"   • {CONFIG['output_path']}/trackable_mapping.json")
if not df_final.empty:
    print(f"   • {CONFIG['output_path']}/flaredown_train.csv")
    print(f"   • {CONFIG['output_path']}/flaredown_validation.csv")
    print(f"   • {CONFIG['output_path']}/flaredown_test.csv")
print(f"   • {CONFIG['reports_path']}/figures/*.png")

if not df_final.empty:
    print(f"\n📊 Final dataset summary:")
    print(f"   • Users: {validation_report['total_users']}")
    print(f"   • Observations: {validation_report['total_observations']}")
    print(f"   • Features: {validation_report['total_features']}")
    print(f"   • Date range: {validation_report['date_range_start']} to {validation_report['date_range_end']}")
    if df_demographics is not None:
        print(f"   • Demographic features: {len(demographic_cols)} ({', '.join(demographic_cols)})")
    if trackable_mapping:
        print(f"   • Symptom features: {validation_report.get('symptom_features', 0)}")
        print(f"   • Condition features: {validation_report.get('condition_features', 0)}")

print("\n✨ You can now use the cleaned datasets for your forecasting analysis!")
print("=" * 70)


✓ Saved: symptom_frequency.png
✓ Saved: condition_frequency.png
----------------------------------------------------------------------
STEP 16: Creating analysis-ready subsets (train/val/test)...
✓ Saved: condition_frequency.png
----------------------------------------------------------------------
STEP 16: Creating analysis-ready subsets (train/val/test)...
✓ Saved train set: 137636 records
✓ Saved validation set: 29631 records
✓ Saved test set: 30796 records
✓ Saved train set: 137636 records
✓ Saved validation set: 29631 records
✓ Saved test set: 30796 records

✅ DATA CLEANING COMPLETE!

End time: 2025-10-24 09:51:36

📁 Output files created:
   • cleaned_data//flaredown_cleaned.csv (full dataset)
   • cleaned_data//flaredown_cleaned.pkl (full dataset)
   • cleaned_data//flaredown_cleaned_sample.xlsx (sample for quick review)
   • cleaned_data//trackable_mapping.json
   • cleaned_data//flaredown_train.csv
   • cleaned_data//flaredown_validation.csv
   • cleaned_data//flaredown_test.cs

# Data Processing Summary

## Pipeline Complete

### Major Changes Implemented

#### 1. Dual Trackable Processing
- **Symptoms** (Top 100): Processed as continuous severity values
  - Lag features (1-7 days)
  - Rolling mean/std (7-day windows)
  - Examples: headache, nausea, fatigue, anxiety, depression
  
- **Conditions** (Top 100): Processed as continuous severity values
  - Lag features (1-7 days)
  - Rolling mean/std (7-day windows)
  - Examples: IBS, fibromyalgia, migraine, Crohn's disease

#### 2. LLM-Powered Intelligence
- **Classification**: Distinguishes acute symptoms from chronic conditions
- **Deduplication**: Merges redundant names (headache/headaches)
- **Auto-Correction**: Swaps misclassified items between categories
  - Items reported in wrong category get automatically reclassified
  - Anxiety and depression are enforced as conditions

#### 3. Enhanced Features
- Demographics preserved (age, sex/gender, location)
- Temporal features (day of week, weekend, month)
- User-specific lag and rolling statistics
- Age-stratified analysis for anxiety and depression

#### 4. Comprehensive Outputs
- `flaredown_cleaned.csv/pkl/xlsx` - Main dataset
- `trackable_mapping.json` - Classification and deduplication mapping
- `flaredown_train/validation/test.csv` - Model-ready splits
- Visualization reports including:
  - Age-based severity analysis
  - Condition prevalence
  - Symptom correlations
  - Temporal coverage

### Pipeline Structure
```
STEP 1-5:   Load, clean, standardize data
STEP 6-9:   Process trackables (symptoms + conditions)
STEP 10:    Identify top 100 symptoms, top 100 conditions
STEP 10A:   LLM classification and deduplication
STEP 10B:   Pivot to wide format (separate symptom/condition columns)
STEP 10C:   Merge demographics
STEP 11:    Feature engineering (lags, rolling stats, temporal)
STEP 12:    Filter users with sufficient data
STEP 13:    Validation report
STEP 14:    Export datasets
STEP 15:    Generate visualizations
STEP 16:    Create train/val/test splits
STEP 17:    Anxiety and depression deep dive analysis
```

### Code Quality
- Logical flow maintained
- Error handling for missing data
- Fallback when LLM unavailable
- Comprehensive logging
- Robust to edge cases

### Next Steps
1. Run the pipeline with your data
2. Review `trackable_mapping.json` to see LLM classifications
3. Check visualizations in `reports/figures/`
4. Use train/validation/test splits for forecasting models

**Ready for analysis**

# Anxiety and Depression Deep Dive Analysis

This section provides comprehensive visualizations and statistics specifically for anxiety and depression symptoms, including:

1. **Prevalence Analysis**: How common are these symptoms across the patient population?
2. **Demographics**: Age, sex, and geographic distributions
3. **Severity Analysis**: Distribution of severity scores and temporal patterns
4. **Comorbidities**: What other symptoms and conditions co-occur?
5. **Correlations**: Which symptoms are most strongly associated?
6. **Temporal Patterns**: Day of week, weekend effects, and trends over time
7. **Patient Clustering**: Distinct patterns of anxiety/depression presentation

In [8]:
# DIAGNOSTIC: Check for anxiety and depression in the dataset
print("=" * 70)
print("DIAGNOSTIC: Searching for anxiety and depression columns")
print("=" * 70)

# Search all columns for anxiety/depression
anxiety_matches = [col for col in df_final.columns if 'anxiety' in col.lower()]
depression_matches = [col for col in df_final.columns if 'depression' in col.lower()]

print(f"\n🔍 Columns containing 'anxiety': {len(anxiety_matches)}")
for col in anxiety_matches:
    print(f"   - {col}")

print(f"\n🔍 Columns containing 'depression': {len(depression_matches)}")
for col in depression_matches:
    print(f"   - {col}")

# Check symptom columns specifically
symptom_cols = [col for col in df_final.columns if col.startswith('symptom_')]
print(f"\n📊 Total symptom columns: {len(symptom_cols)}")
print(f"   Sample symptom columns: {symptom_cols[:10]}")

# Check if anxiety/depression exist in any form
all_symptom_names = [col.replace('symptom_', '') for col in symptom_cols]
anxiety_in_symptoms = [name for name in all_symptom_names if 'anxiety' in name.lower() or 'anxious' in name.lower()]
depression_in_symptoms = [name for name in all_symptom_names if 'depression' in name.lower() or 'depressed' in name.lower()]

print(f"\n✓ Anxiety-related symptoms found: {anxiety_in_symptoms}")
print(f"✓ Depression-related symptoms found: {depression_in_symptoms}")

# Check condition columns too
condition_cols = [col for col in df_final.columns if col.startswith('condition_')]
print(f"\n📊 Total condition columns: {len(condition_cols)}")
all_condition_names = [col.replace('condition_', '') for col in condition_cols]
anxiety_in_conditions = [name for name in all_condition_names if 'anxiety' in name.lower()]
depression_in_conditions = [name for name in all_condition_names if 'depression' in name.lower()]

print(f"✓ Anxiety-related conditions found: {anxiety_in_conditions}")
print(f"✓ Depression-related conditions found: {depression_in_conditions}")

print("\n" + "=" * 70)

DIAGNOSTIC: Searching for anxiety and depression columns

🔍 Columns containing 'anxiety': 20
   - condition_anxiety
   - condition_depression/anxiety
   - condition_anxiety_lag1
   - condition_anxiety_lag2
   - condition_anxiety_lag3
   - condition_anxiety_lag4
   - condition_anxiety_lag5
   - condition_anxiety_lag6
   - condition_anxiety_lag7
   - condition_depression/anxiety_lag1
   - condition_depression/anxiety_lag2
   - condition_depression/anxiety_lag3
   - condition_depression/anxiety_lag4
   - condition_depression/anxiety_lag5
   - condition_depression/anxiety_lag6
   - condition_depression/anxiety_lag7
   - condition_anxiety_rolling_mean_7d
   - condition_anxiety_rolling_std_7d
   - condition_depression/anxiety_rolling_mean_7d
   - condition_depression/anxiety_rolling_std_7d

🔍 Columns containing 'depression': 20
   - condition_depression
   - condition_depression/anxiety
   - condition_depression_lag1
   - condition_depression_lag2
   - condition_depression_lag3
   - conditio

In [9]:
# STEP 17: ANXIETY & DEPRESSION COMPREHENSIVE ANALYSIS
print("\n" + "=" * 70)
print("STEP 17: ANXIETY & DEPRESSION COMPREHENSIVE ANALYSIS")
print("=" * 70)

import seaborn as sns
from scipy import stats
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler

# Set seaborn style for better-looking plots
sns.set_style("whitegrid")
sns.set_palette("husl")

# Create anxiety/depression specific figure directory
ad_fig_path = os.path.join(CONFIG['reports_path'], "figures/anxiety_depression/")
Path(ad_fig_path).mkdir(parents=True, exist_ok=True)

# Identify anxiety and depression columns (check both symptoms AND conditions)
anxiety_col = None
depression_col = None

# First try to find as symptoms
for col in df_final.columns:
    if 'anxiety' in col.lower() and 'symptom_' in col and 'lag' not in col and 'rolling' not in col:
        anxiety_col = col
        break

# If not found in symptoms, check conditions
if anxiety_col is None:
    for col in df_final.columns:
        if 'anxiety' in col.lower() and 'condition_' in col and 'lag' not in col and 'rolling' not in col:
            anxiety_col = col
            break

# Try to find depression as symptom
for col in df_final.columns:
    if ('depression' in col.lower() or 'depressed' in col.lower()) and 'symptom_' in col and 'lag' not in col and 'rolling' not in col:
        depression_col = col
        break

# If not found in symptoms, check conditions
if depression_col is None:
    for col in df_final.columns:
        if ('depression' in col.lower() or 'depressed' in col.lower()) and 'condition_' in col and 'lag' not in col and 'rolling' not in col:
            depression_col = col
            break

if anxiety_col is None or depression_col is None:
    print("⚠️  Warning: Could not find anxiety and/or depression columns")
    print(f"   Anxiety column: {anxiety_col}")
    print(f"   Depression column: {depression_col}")
    print("   Skipping anxiety/depression analysis...")
else:
    print(f"✓ Found columns:")
    print(f"   Anxiety: {anxiety_col}")
    print(f"   Depression: {depression_col}")
    
    # Create analysis dataset (non-null values)
    df_ad = df_final[[user_col, date_col, anxiety_col, depression_col] + 
                     (demographic_cols if df_demographics is not None else [])].copy()
    
    # Add all symptom and condition columns for comorbidity analysis
    all_symptoms = [col for col in df_final.columns if col.startswith('symptom_') and 'lag' not in col and 'rolling' not in col]
    all_conditions = [col for col in df_final.columns if col.startswith('condition_') and 'lag' not in col and 'rolling' not in col]
    
    for col in all_symptoms + all_conditions:
        if col not in df_ad.columns:
            df_ad[col] = df_final[col]
    
    print(f"\n📊 Dataset for analysis: {df_ad.shape}")
    print(f"   Total observations: {len(df_ad)}")
    print(f"   Unique users: {df_ad[user_col].nunique()}")
    
    # ============================================================================
    # REPORT 1: PREVALENCE ANALYSIS
    # ============================================================================
    print("\n" + "-" * 70)
    print("REPORT 1: Prevalence Analysis")
    print("-" * 70)
    
    # Calculate prevalence metrics
    anxiety_users = df_ad[df_ad[anxiety_col] > 0][user_col].nunique()
    depression_users = df_ad[df_ad[depression_col] > 0][user_col].nunique()
    both_users = df_ad[(df_ad[anxiety_col] > 0) & (df_ad[depression_col] > 0)][user_col].nunique()
    total_users = df_ad[user_col].nunique()
    
    anxiety_obs = (df_ad[anxiety_col] > 0).sum()
    depression_obs = (df_ad[depression_col] > 0).sum()
    both_obs = ((df_ad[anxiety_col] > 0) & (df_ad[depression_col] > 0)).sum()
    total_obs = len(df_ad)
    
    print(f"✓ Prevalence Metrics:")
    print(f"   Users with anxiety: {anxiety_users}/{total_users} ({anxiety_users/total_users*100:.1f}%)")
    print(f"   Users with depression: {depression_users}/{total_users} ({depression_users/total_users*100:.1f}%)")
    print(f"   Users with both: {both_users}/{total_users} ({both_users/total_users*100:.1f}%)")
    print(f"   Observations with anxiety: {anxiety_obs}/{total_obs} ({anxiety_obs/total_obs*100:.1f}%)")
    print(f"   Observations with depression: {depression_obs}/{total_obs} ({depression_obs/total_obs*100:.1f}%)")
    
    # Create prevalence visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # User-level prevalence
    categories = ['Anxiety\nOnly', 'Depression\nOnly', 'Both', 'Neither']
    anxiety_only = anxiety_users - both_users
    depression_only = depression_users - both_users
    neither = total_users - anxiety_users - depression_users + both_users
    counts = [anxiety_only, depression_only, both_users, neither]
    colors = ['#FF6B6B', '#4ECDC4', '#95E1D3', '#F3F3F3']
    
    axes[0].bar(categories, counts, color=colors, edgecolor='black', linewidth=1.5)
    axes[0].set_ylabel('Number of Users', fontsize=12, fontweight='bold')
    axes[0].set_title('User-Level Prevalence', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add percentage labels on bars
    for i, (cat, count) in enumerate(zip(categories, counts)):
        pct = count / total_users * 100
        axes[0].text(i, count + max(counts)*0.02, f'{count}\n({pct:.1f}%)', 
                    ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Observation-level prevalence (Venn diagram style)
    from matplotlib.patches import Circle
    axes[1].set_xlim(0, 10)
    axes[1].set_ylim(0, 10)
    axes[1].set_aspect('equal')
    
    # Draw circles
    circle1 = Circle((3.5, 5), 2.5, color='#FF6B6B', alpha=0.3, label='Anxiety')
    circle2 = Circle((6.5, 5), 2.5, color='#4ECDC4', alpha=0.3, label='Depression')
    axes[1].add_patch(circle1)
    axes[1].add_patch(circle2)
    
    # Add text labels
    axes[1].text(2.5, 5, f'{anxiety_obs-both_obs}\nAnxiety\nOnly', 
                ha='center', va='center', fontweight='bold', fontsize=10)
    axes[1].text(7.5, 5, f'{depression_obs-both_obs}\nDepression\nOnly', 
                ha='center', va='center', fontweight='bold', fontsize=10)
    axes[1].text(5, 5, f'{both_obs}\nBoth', 
                ha='center', va='center', fontweight='bold', fontsize=11)
    
    axes[1].set_title('Observation-Level Co-occurrence', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "01_prevalence.png"), dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 01_prevalence.png")


STEP 17: ANXIETY & DEPRESSION COMPREHENSIVE ANALYSIS
✓ Found columns:
   Anxiety: condition_anxiety
   Depression: condition_depression

📊 Dataset for analysis: (198063, 149)
   Total observations: 198063
   Unique users: 2221

----------------------------------------------------------------------
REPORT 1: Prevalence Analysis
----------------------------------------------------------------------
✓ Prevalence Metrics:
   Users with anxiety: 933/2221 (42.0%)
   Users with depression: 790/2221 (35.6%)
   Users with both: 578/2221 (26.0%)
   Observations with anxiety: 54143/198063 (27.3%)
   Observations with depression: 39332/198063 (19.9%)
✓ Saved: 01_prevalence.png
✓ Prevalence Metrics:
   Users with anxiety: 933/2221 (42.0%)
   Users with depression: 790/2221 (35.6%)
   Users with both: 578/2221 (26.0%)
   Observations with anxiety: 54143/198063 (27.3%)
   Observations with depression: 39332/198063 (19.9%)
✓ Saved: 01_prevalence.png


In [10]:
# REPORT 2: DEMOGRAPHICS ANALYSIS (only runs if anxiety/depression analysis is active)
if anxiety_col is not None and depression_col is not None:
    print("\n" + "-" * 70)
    print("REPORT 2: Demographics Analysis")
    print("-" * 70)
    
    if df_demographics is not None and len(demographic_cols) > 0:
        # Get user-level data (one row per user with their max severity)
        df_user_summary = df_ad.groupby(user_col).agg({
            anxiety_col: 'max',
            depression_col: 'max',
            **{col: 'first' for col in demographic_cols if col in df_ad.columns}
        }).reset_index()
        
        # Create demographic visualizations
        fig = plt.figure(figsize=(18, 12))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Age analysis (if available)
        age_col = next((col for col in demographic_cols if 'age' in col.lower()), None)
        if age_col and age_col in df_user_summary.columns:
            # Age distribution by symptom presence
            ax1 = fig.add_subplot(gs[0, :2])
            
            df_age = df_user_summary.dropna(subset=[age_col])
            df_age['group'] = 'Neither'
            df_age.loc[df_age[anxiety_col] > 0, 'group'] = 'Anxiety'
            df_age.loc[df_age[depression_col] > 0, 'group'] = 'Depression'
            df_age.loc[(df_age[anxiety_col] > 0) & (df_age[depression_col] > 0), 'group'] = 'Both'
            
            for group, color in zip(['Neither', 'Anxiety', 'Depression', 'Both'], 
                                   ['#F3F3F3', '#FF6B6B', '#4ECDC4', '#95E1D3']):
                data = df_age[df_age['group'] == group][age_col]
                if len(data) > 0:
                    ax1.hist(data, bins=20, alpha=0.6, label=f'{group} (n={len(data)})', 
                            color=color, edgecolor='black', linewidth=0.5)
            
            ax1.set_xlabel('Age', fontsize=12, fontweight='bold')
            ax1.set_ylabel('Number of Users', fontsize=12, fontweight='bold')
            ax1.set_title('Age Distribution by Symptom Group', fontsize=14, fontweight='bold')
            ax1.legend(loc='upper right')
            ax1.grid(axis='y', alpha=0.3)
            
            # Age vs severity scatter
            ax2 = fig.add_subplot(gs[0, 2])
            ax2.scatter(df_age[age_col], df_age[anxiety_col], alpha=0.5, 
                       color='#FF6B6B', s=30, label='Anxiety')
            ax2.scatter(df_age[age_col], df_age[depression_col], alpha=0.5, 
                       color='#4ECDC4', s=30, label='Depression')
            ax2.set_xlabel('Age', fontsize=11, fontweight='bold')
            ax2.set_ylabel('Max Severity', fontsize=11, fontweight='bold')
            ax2.set_title('Age vs Severity', fontsize=12, fontweight='bold')
            ax2.legend()
            ax2.grid(alpha=0.3)
            
            # Calculate correlations
            anxiety_corr = df_age[df_age[anxiety_col] > 0][[age_col, anxiety_col]].corr().iloc[0, 1] if len(df_age[df_age[anxiety_col] > 0]) > 1 else 0
            depression_corr = df_age[df_age[depression_col] > 0][[age_col, depression_col]].corr().iloc[0, 1] if len(df_age[df_age[depression_col] > 0]) > 1 else 0
            print(f"   Age correlations:")
            print(f"     Anxiety: r={anxiety_corr:.3f}")
            print(f"     Depression: r={depression_corr:.3f}")
        
        # Sex/Gender analysis (if available)
        sex_col = next((col for col in demographic_cols if 'sex' in col.lower() or 'gender' in col.lower()), None)
        if sex_col and sex_col in df_user_summary.columns:
            ax3 = fig.add_subplot(gs[1, :2])
            
            df_sex = df_user_summary.dropna(subset=[sex_col])
            sex_counts = df_sex.groupby(sex_col).agg({
                anxiety_col: lambda x: (x > 0).sum(),
                depression_col: lambda x: (x > 0).sum(),
                user_col: 'count'
            }).reset_index()
            sex_counts.columns = [sex_col, 'Anxiety', 'Depression', 'Total']
            
            x = range(len(sex_counts))
            width = 0.25
            
            ax3.bar([i - width for i in x], sex_counts['Anxiety'] / sex_counts['Total'] * 100, 
                   width, label='Anxiety', color='#FF6B6B', edgecolor='black')
            ax3.bar([i for i in x], sex_counts['Depression'] / sex_counts['Total'] * 100, 
                   width, label='Depression', color='#4ECDC4', edgecolor='black')
            ax3.bar([i + width for i in x], 
                   sex_counts.apply(lambda row: ((df_sex[df_sex[sex_col] == row[sex_col]][anxiety_col] > 0) & 
                                                 (df_sex[df_sex[sex_col] == row[sex_col]][depression_col] > 0)).sum() / 
                                                row['Total'] * 100 if row['Total'] > 0 else 0, axis=1),
                   width, label='Both', color='#95E1D3', edgecolor='black')
            
            ax3.set_ylabel('Prevalence (%)', fontsize=12, fontweight='bold')
            ax3.set_title(f'Prevalence by {sex_col.replace("_", " ").title()}', fontsize=14, fontweight='bold')
            ax3.set_xticks(x)
            ax3.set_xticklabels(sex_counts[sex_col])
            ax3.legend()
            ax3.grid(axis='y', alpha=0.3)
            
            # Average severity by sex
            ax4 = fig.add_subplot(gs[1, 2])
            sex_severity = df_sex[df_sex[anxiety_col] > 0].groupby(sex_col)[anxiety_col].mean()
            sex_severity_dep = df_sex[df_sex[depression_col] > 0].groupby(sex_col)[depression_col].mean()
            
            # Align both series to have the same index (sex categories)
            all_sex_categories = sorted(set(sex_severity.index) | set(sex_severity_dep.index))
            sex_severity = sex_severity.reindex(all_sex_categories, fill_value=0)
            sex_severity_dep = sex_severity_dep.reindex(all_sex_categories, fill_value=0)
            
            x_pos = range(len(sex_severity))
            ax4.bar([i - 0.2 for i in x_pos], sex_severity, 0.4, 
                   label='Anxiety', color='#FF6B6B', edgecolor='black')
            ax4.bar([i + 0.2 for i in x_pos], sex_severity_dep, 0.4, 
                   label='Depression', color='#4ECDC4', edgecolor='black')
            ax4.set_ylabel('Avg Severity', fontsize=11, fontweight='bold')
            ax4.set_title('Avg Severity by Sex', fontsize=12, fontweight='bold')
            ax4.set_xticks(x_pos)
            ax4.set_xticklabels(sex_severity.index)
            ax4.legend()
            ax4.grid(axis='y', alpha=0.3)
        
        # Country/Location analysis (if available)
        country_col = next((col for col in demographic_cols if 'country' in col.lower() or 'location' in col.lower()), None)
        if country_col and country_col in df_user_summary.columns:
            ax5 = fig.add_subplot(gs[2, :])
            
            # Get top 10 countries by user count
            top_countries = df_user_summary[country_col].value_counts().head(10).index
            df_country = df_user_summary[df_user_summary[country_col].isin(top_countries)]
            
            country_stats = df_country.groupby(country_col).agg({
                anxiety_col: lambda x: (x > 0).sum() / len(x) * 100,
                depression_col: lambda x: (x > 0).sum() / len(x) * 100,
                user_col: 'count'
            }).reset_index()
            country_stats.columns = [country_col, 'Anxiety %', 'Depression %', 'Users']
            country_stats = country_stats.sort_values('Users', ascending=False)
            
            x = range(len(country_stats))
            width = 0.35
            
            ax5.bar([i - width/2 for i in x], country_stats['Anxiety %'], 
                   width, label='Anxiety', color='#FF6B6B', edgecolor='black')
            ax5.bar([i + width/2 for i in x], country_stats['Depression %'], 
                   width, label='Depression', color='#4ECDC4', edgecolor='black')
            
            ax5.set_ylabel('Prevalence (%)', fontsize=12, fontweight='bold')
            ax5.set_title('Prevalence by Country/Location (Top 10)', fontsize=14, fontweight='bold')
            ax5.set_xticks(x)
            ax5.set_xticklabels(country_stats[country_col], rotation=45, ha='right')
            ax5.legend()
            ax5.grid(axis='y', alpha=0.3)
            
            # Add user counts as text
            for i, users in enumerate(country_stats['Users']):
                ax5.text(i, -5, f'n={users}', ha='center', va='top', fontsize=9)
        
        plt.savefig(os.path.join(ad_fig_path, "02_demographics.png"), dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Saved: 02_demographics.png")
    else:
        print("   No demographic data available, skipping demographics report")


----------------------------------------------------------------------
REPORT 2: Demographics Analysis
----------------------------------------------------------------------
   Age correlations:
     Anxiety: r=-0.063
     Depression: r=-0.116
✓ Saved: 02_demographics.png
✓ Saved: 02_demographics.png


In [11]:
# REPORT 3: SEVERITY ANALYSIS & TEMPORAL PATTERNS
if anxiety_col is not None and depression_col is not None:
    print("\n" + "-" * 70)
    print("REPORT 3: Severity Distribution & Temporal Patterns")
    print("-" * 70)
    
    def categorize_severity(val):
        if val == 0:
            return 'None'
        elif val <= 1:
            return 'Mild'
        elif val <= 2:
            return 'Moderate'
        else:
            return 'Severe'
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Distribution of severity scores
    ax1 = axes[0, 0]
    anxiety_vals = df_ad[df_ad[anxiety_col] > 0][anxiety_col]
    depression_vals = df_ad[df_ad[depression_col] > 0][depression_col]
    
    ax1.hist(anxiety_vals, bins=20, alpha=0.6, label=f'Anxiety (n={len(anxiety_vals)})', 
            color='#FF6B6B', edgecolor='black', density=True)
    ax1.hist(depression_vals, bins=20, alpha=0.6, label=f'Depression (n={len(depression_vals)})', 
            color='#4ECDC4', edgecolor='black', density=True)
    ax1.set_xlabel('Severity Score', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Density', fontsize=11, fontweight='bold')
    ax1.set_title('Severity Distribution', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    print(f"   Anxiety severity - Mean: {anxiety_vals.mean():.2f}, Median: {anxiety_vals.median():.2f}, Std: {anxiety_vals.std():.2f}")
    print(f"   Depression severity - Mean: {depression_vals.mean():.2f}, Median: {depression_vals.median():.2f}, Std: {depression_vals.std():.2f}")
    
    # Box plots
    ax2 = axes[0, 1]
    bp_data = [anxiety_vals, depression_vals]
    bp = ax2.boxplot(bp_data, labels=['Anxiety', 'Depression'], patch_artist=True,
                     boxprops=dict(facecolor='lightblue', color='black'),
                     medianprops=dict(color='red', linewidth=2),
                     whiskerprops=dict(color='black'),
                     capprops=dict(color='black'))
    bp['boxes'][0].set_facecolor('#FF6B6B')
    bp['boxes'][1].set_facecolor('#4ECDC4')
    ax2.set_ylabel('Severity Score', fontsize=11, fontweight='bold')
    ax2.set_title('Severity Distribution (Boxplot)', fontsize=12, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Severity categories
    ax3 = axes[0, 2]
    anxiety_cats = df_ad[anxiety_col].apply(categorize_severity).value_counts()
    depression_cats = df_ad[depression_col].apply(categorize_severity).value_counts()
    
    categories = ['None', 'Mild', 'Moderate', 'Severe']
    anxiety_pcts = [anxiety_cats.get(cat, 0) / len(df_ad) * 100 for cat in categories]
    depression_pcts = [depression_cats.get(cat, 0) / len(df_ad) * 100 for cat in categories]
    
    x = range(len(categories))
    width = 0.35
    ax3.bar([i - width/2 for i in x], anxiety_pcts, width, label='Anxiety', 
           color='#FF6B6B', edgecolor='black')
    ax3.bar([i + width/2 for i in x], depression_pcts, width, label='Depression', 
           color='#4ECDC4', edgecolor='black')
    ax3.set_ylabel('Percentage of Observations', fontsize=11, fontweight='bold')
    ax3.set_title('Severity Categories', fontsize=12, fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(categories)
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
    
    # Day of week patterns
    ax4 = axes[1, 0]
    df_ad['day_of_week'] = pd.to_datetime(df_ad[date_col]).dt.dayofweek
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    anxiety_by_day = df_ad[df_ad[anxiety_col] > 0].groupby('day_of_week')[anxiety_col].mean()
    depression_by_day = df_ad[df_ad[depression_col] > 0].groupby('day_of_week')[depression_col].mean()
    
    ax4.plot(range(7), [anxiety_by_day.get(i, 0) for i in range(7)], 
            marker='o', linewidth=2, markersize=8, label='Anxiety', color='#FF6B6B')
    ax4.plot(range(7), [depression_by_day.get(i, 0) for i in range(7)], 
            marker='s', linewidth=2, markersize=8, label='Depression', color='#4ECDC4')
    ax4.set_xlabel('Day of Week', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Average Severity', fontsize=11, fontweight='bold')
    ax4.set_title('Day of Week Patterns', fontsize=12, fontweight='bold')
    ax4.set_xticks(range(7))
    ax4.set_xticklabels(day_names)
    ax4.legend()
    ax4.grid(alpha=0.3)
    
    # Weekend vs Weekday
    ax5 = axes[1, 1]
    df_ad['is_weekend'] = df_ad['day_of_week'].isin([5, 6])
    
    anxiety_weekday = df_ad[(df_ad[anxiety_col] > 0) & (~df_ad['is_weekend'])][anxiety_col].mean()
    anxiety_weekend = df_ad[(df_ad[anxiety_col] > 0) & (df_ad['is_weekend'])][anxiety_col].mean()
    depression_weekday = df_ad[(df_ad[depression_col] > 0) & (~df_ad['is_weekend'])][depression_col].mean()
    depression_weekend = df_ad[(df_ad[depression_col] > 0) & (df_ad['is_weekend'])][depression_col].mean()
    
    x = [0, 1]
    width = 0.35
    ax5.bar([i - width/2 for i in x], [anxiety_weekday, anxiety_weekend], width, 
           label='Anxiety', color='#FF6B6B', edgecolor='black')
    ax5.bar([i + width/2 for i in x], [depression_weekday, depression_weekend], width, 
           label='Depression', color='#4ECDC4', edgecolor='black')
    ax5.set_ylabel('Average Severity', fontsize=11, fontweight='bold')
    ax5.set_title('Weekday vs Weekend', fontsize=12, fontweight='bold')
    ax5.set_xticks(x)
    ax5.set_xticklabels(['Weekday', 'Weekend'])
    ax5.legend()
    ax5.grid(axis='y', alpha=0.3)
    
    print(f"   Weekend effect:")
    print(f"     Anxiety: Weekday={anxiety_weekday:.2f}, Weekend={anxiety_weekend:.2f}")
    print(f"     Depression: Weekday={depression_weekday:.2f}, Weekend={depression_weekend:.2f}")
    
    # Temporal trend (by days since first record per user)
    ax6 = axes[1, 2]
    
    # Calculate days since first record for each user
    df_ad_temp = df_ad.copy()
    df_ad_temp['first_date'] = df_ad_temp.groupby(user_col)[date_col].transform('min')
    df_ad_temp['days_since_first'] = (pd.to_datetime(df_ad_temp[date_col]) - 
                                       pd.to_datetime(df_ad_temp['first_date'])).dt.days
    
    # Group by time bins (e.g., 14-day intervals) and calculate mean severity
    bin_size = 14  # 2-week intervals
    df_ad_temp['time_bin'] = (df_ad_temp['days_since_first'] // bin_size) * bin_size
    
    # Calculate mean severity for each time bin
    anxiety_temporal = df_ad_temp[df_ad_temp[anxiety_col] > 0].groupby('time_bin')[anxiety_col].mean()
    depression_temporal = df_ad_temp[df_ad_temp[depression_col] > 0].groupby('time_bin')[depression_col].mean()
    
    if len(anxiety_temporal) > 1 or len(depression_temporal) > 1:
        if len(anxiety_temporal) > 0:
            ax6.plot(anxiety_temporal.index, anxiety_temporal.values, 
                    marker='o', linewidth=2, markersize=5, label='Anxiety', 
                    color='#FF6B6B', alpha=0.7)
        if len(depression_temporal) > 0:
            ax6.plot(depression_temporal.index, depression_temporal.values, 
                    marker='s', linewidth=2, markersize=5, label='Depression', 
                    color='#4ECDC4', alpha=0.7)
        ax6.set_xlabel('Days Since First Record', fontsize=11, fontweight='bold')
        ax6.set_ylabel('Average Severity', fontsize=11, fontweight='bold')
        ax6.set_title('Temporal Trend (Aligned by User Start)', fontsize=12, fontweight='bold')
        ax6.legend()
        ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "03_severity_temporal.png"), dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 03_severity_temporal.png")


----------------------------------------------------------------------
REPORT 3: Severity Distribution & Temporal Patterns
----------------------------------------------------------------------
   Anxiety severity - Mean: 1.95, Median: 2.00, Std: 0.94
   Depression severity - Mean: 1.90, Median: 2.00, Std: 0.95
   Weekend effect:
     Anxiety: Weekday=1.97, Weekend=1.90
     Depression: Weekday=1.90, Weekend=1.88
   Weekend effect:
     Anxiety: Weekday=1.97, Weekend=1.90
     Depression: Weekday=1.90, Weekend=1.88
✓ Saved: 03_severity_temporal.png
✓ Saved: 03_severity_temporal.png


In [12]:
# REPORTS 4-7: COMORBIDITIES, CORRELATIONS, RELATIONSHIP & USER PATTERNS
if anxiety_col is not None and depression_col is not None:
    # ========================================================================
    # REPORT 4: COMORBIDITY ANALYSIS
    # ========================================================================
    print("\n" + "-" * 70)
    print("REPORT 4: Comorbidity Analysis")
    print("-" * 70)
    
    # Find observations with anxiety or depression
    df_anxiety = df_ad[df_ad[anxiety_col] > 0].copy()
    df_depression = df_ad[df_ad[depression_col] > 0].copy()
    
    # Calculate co-occurrence rates for other symptoms
    other_symptoms = [col for col in all_symptoms if col not in [anxiety_col, depression_col]]
    other_conditions = all_conditions[:30] if len(all_conditions) > 30 else all_conditions
    
    # For anxiety
    anxiety_comorbid = {}
    for col in other_symptoms + other_conditions:
        if col in df_anxiety.columns:
            co_occur = (df_anxiety[col] > 0).sum()
            pct = co_occur / len(df_anxiety) * 100
            anxiety_comorbid[col] = pct
    
    # For depression
    depression_comorbid = {}
    for col in other_symptoms + other_conditions:
        if col in df_depression.columns:
            co_occur = (df_depression[col] > 0).sum()
            pct = co_occur / len(df_depression) * 100
            depression_comorbid[col] = pct
    
    # Sort and get top 15
    anxiety_top = sorted(anxiety_comorbid.items(), key=lambda x: x[1], reverse=True)[:15]
    depression_top = sorted(depression_comorbid.items(), key=lambda x: x[1], reverse=True)[:15]
    
    print(f"   Top comorbidities with anxiety:")
    for symptom, pct in anxiety_top[:5]:
        clean_name = symptom.replace('symptom_', '').replace('condition_', '').replace('_', ' ').title()
        print(f"     {clean_name}: {pct:.1f}%")
    
    print(f"   Top comorbidities with depression:")
    for symptom, pct in depression_top[:5]:
        clean_name = symptom.replace('symptom_', '').replace('condition_', '').replace('_', ' ').title()
        print(f"     {clean_name}: {pct:.1f}%")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Anxiety comorbidities
    ax1 = axes[0]
    names = [s[0].replace('symptom_', '').replace('condition_', '').replace('_', ' ').title() for s in anxiety_top]
    values = [s[1] for s in anxiety_top]
    colors_list = ['#FF8C94' if 'symptom_' in anxiety_top[i][0] else '#FFC994' for i in range(len(names))]
    
    y_pos = range(len(names))
    ax1.barh(y_pos, values, color=colors_list, edgecolor='black')
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(names, fontsize=10)
    ax1.set_xlabel('Co-occurrence Rate (%)', fontsize=12, fontweight='bold')
    ax1.set_title(f'Top Comorbidities with Anxiety\n(n={len(df_anxiety)} observations)', 
                 fontsize=13, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    ax1.invert_yaxis()
    
    for i, v in enumerate(values):
        ax1.text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold', fontsize=9)
    
    # Depression comorbidities
    ax2 = axes[1]
    names = [s[0].replace('symptom_', '').replace('condition_', '').replace('_', ' ').title() for s in depression_top]
    values = [s[1] for s in depression_top]
    colors_list = ['#8CD4FF' if 'symptom_' in depression_top[i][0] else '#94FFD6' for i in range(len(names))]
    
    y_pos = range(len(names))
    ax2.barh(y_pos, values, color=colors_list, edgecolor='black')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(names, fontsize=10)
    ax2.set_xlabel('Co-occurrence Rate (%)', fontsize=12, fontweight='bold')
    ax2.set_title(f'Top Comorbidities with Depression\n(n={len(df_depression)} observations)', 
                 fontsize=13, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    ax2.invert_yaxis()
    
    for i, v in enumerate(values):
        ax2.text(v + 1, i, f'{v:.1f}%', va='center', fontweight='bold', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "04_comorbidities.png"), dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 04_comorbidities.png")
    
    # ========================================================================
    # REPORT 5: CORRELATION ANALYSIS
    # ========================================================================
    print("\n" + "-" * 70)
    print("REPORT 5: Correlation Analysis")
    print("-" * 70)
    
    # Select top symptoms for correlation analysis
    top_20_symptoms = [col for col in all_symptoms if col not in [anxiety_col, depression_col]][:20]
    correlation_cols = [anxiety_col, depression_col] + top_20_symptoms
    
    # Calculate correlation matrix
    df_corr = df_ad[correlation_cols].copy()
    corr_matrix = df_corr.corr()
    
    # Get anxiety and depression correlations
    anxiety_corrs = corr_matrix[anxiety_col].drop(anxiety_col).sort_values(ascending=False)
    depression_corrs = corr_matrix[depression_col].drop(depression_col).sort_values(ascending=False)
    
    print(f"   Anxiety-Depression correlation: r={corr_matrix.loc[anxiety_col, depression_col]:.3f}")
    print(f"   Top correlates with anxiety:")
    for idx, val in anxiety_corrs.head(5).items():
        clean_name = idx.replace('symptom_', '').replace('_', ' ').title()
        print(f"     {clean_name}: r={val:.3f}")
    
    print(f"   Top correlates with depression:")
    for idx, val in depression_corrs.head(5).items():
        clean_name = idx.replace('symptom_', '').replace('_', ' ').title()
        print(f"     {clean_name}: r={val:.3f}")
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # Full correlation heatmap
    ax1 = axes[0]
    sns.heatmap(corr_matrix, annot=False, cmap='RdBu_r', center=0, 
               vmin=-1, vmax=1, ax=ax1, cbar_kws={'label': 'Correlation'})
    ax1.set_title('Symptom Correlation Matrix', fontsize=13, fontweight='bold')
    
    labels = [col.replace('symptom_', '').replace('condition_', '').replace('_', ' ').title() for col in correlation_cols]
    ax1.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax1.set_yticklabels(labels, rotation=0, fontsize=8)
    
    # Anxiety correlations bar chart
    ax2 = axes[1]
    top_anxiety_corrs = anxiety_corrs.head(10)
    names = [idx.replace('symptom_', '').replace('condition_', '').replace('_', ' ').title() for idx in top_anxiety_corrs.index]
    values = top_anxiety_corrs.values
    colors_bar = ['#FF6B6B' if v > 0.3 else '#FFB6B6' for v in values]
    
    y_pos = range(len(names))
    ax2.barh(y_pos, values, color=colors_bar, edgecolor='black')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(names, fontsize=10)
    ax2.set_xlabel('Correlation Coefficient', fontsize=11, fontweight='bold')
    ax2.set_title('Top Correlations with Anxiety', fontsize=12, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    ax2.invert_yaxis()
    
    for i, v in enumerate(values):
        ax2.text(v + 0.01, i, f'{v:.2f}', va='center', fontweight='bold', fontsize=9)
    
    # Depression correlations bar chart
    ax3 = axes[2]
    top_depression_corrs = depression_corrs.head(10)
    names = [idx.replace('symptom_', '').replace('condition_', '').replace('_', ' ').title() for idx in top_depression_corrs.index]
    values = top_depression_corrs.values
    colors_bar = ['#4ECDC4' if v > 0.3 else '#9CE8E0' for v in values]
    
    y_pos = range(len(names))
    ax3.barh(y_pos, values, color=colors_bar, edgecolor='black')
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels(names, fontsize=10)
    ax3.set_xlabel('Correlation Coefficient', fontsize=11, fontweight='bold')
    ax3.set_title('Top Correlations with Depression', fontsize=12, fontweight='bold')
    ax3.grid(axis='x', alpha=0.3)
    ax3.invert_yaxis()
    
    for i, v in enumerate(values):
        ax3.text(v + 0.01, i, f'{v:.2f}', va='center', fontweight='bold', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "05_correlations.png"), dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 05_correlations.png")
    
    # ========================================================================
    # REPORT 6: ANXIETY-DEPRESSION RELATIONSHIP
    # ========================================================================
    print("\n" + "-" * 70)
    print("REPORT 6: Anxiety vs Depression Relationship")
    print("-" * 70)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Scatter plot with density
    ax1 = axes[0, 0]
    df_both = df_ad[(df_ad[anxiety_col] > 0) | (df_ad[depression_col] > 0)]
    
    scatter = ax1.scatter(df_both[anxiety_col], df_both[depression_col], 
                         alpha=0.3, s=20, c='#9B59B6')
    ax1.set_xlabel('Anxiety Severity', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Depression Severity', fontsize=12, fontweight='bold')
    ax1.set_title(f'Anxiety vs Depression Scatter\n(r={corr_matrix.loc[anxiety_col, depression_col]:.3f})', 
                 fontsize=13, fontweight='bold')
    ax1.grid(alpha=0.3)
    
    # Add regression line
    valid_data = df_both[[anxiety_col, depression_col]].dropna()
    if len(valid_data) > 10:
        slope, intercept, r_value, p_value, std_err = linregress(valid_data[anxiety_col], 
                                                                  valid_data[depression_col])
        x_line = np.linspace(valid_data[anxiety_col].min(), valid_data[anxiety_col].max(), 100)
        y_line = slope * x_line + intercept
        ax1.plot(x_line, y_line, 'r--', linewidth=2, label=f'Linear fit (R²={r_value**2:.3f})')
        ax1.legend()
    
    # Joint distribution (2D histogram) - WITH FIX FOR NaN VALUES
    ax2 = axes[0, 1]
    # Remove NaN values and ensure we have valid data
    df_both_valid = df_both[[anxiety_col, depression_col]].dropna()
    df_both_valid = df_both_valid[(df_both_valid[anxiety_col].notna()) & (df_both_valid[depression_col].notna())]
    
    if len(df_both_valid) > 10 and df_both_valid[anxiety_col].std() > 0 and df_both_valid[depression_col].std() > 0:
        h = ax2.hist2d(df_both_valid[anxiety_col], df_both_valid[depression_col], 
                       bins=20, cmap='YlOrRd', cmin=1)
        plt.colorbar(h[3], ax=ax2, label='Count')
    else:
        # If not enough valid data, show message
        ax2.text(0.5, 0.5, 'Insufficient data\nfor 2D histogram', 
                ha='center', va='center', fontsize=12, transform=ax2.transAxes)
    
    ax2.set_xlabel('Anxiety Severity', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Depression Severity', fontsize=12, fontweight='bold')
    ax2.set_title('Joint Distribution (2D Histogram)', fontsize=13, fontweight='bold')
    
    # Severity concordance
    ax3 = axes[1, 0]
    
    # Define severity categorization function
    def categorize_severity(val):
        if val == 0:
            return 'None'
        elif val <= 1:
            return 'Mild'
        elif val <= 2:
            return 'Moderate'
        else:
            return 'Severe'
    
    df_both_copy = df_both.copy()
    df_both_copy['anxiety_cat'] = df_both_copy[anxiety_col].apply(categorize_severity)
    df_both_copy['depression_cat'] = df_both_copy[depression_col].apply(categorize_severity)
    df_both_copy['concordant'] = df_both_copy['anxiety_cat'] == df_both_copy['depression_cat']
    
    concordance = df_both_copy['concordant'].value_counts()
    labels = ['Concordant\n(Same Severity)', 'Discordant\n(Different Severity)']
    sizes = [concordance.get(True, 0), concordance.get(False, 0)]
    colors_pie = ['#95E1D3', '#FFB6B6']
    
    wedges, texts, autotexts = ax3.pie(sizes, labels=labels, autopct='%1.1f%%', 
                                        colors=colors_pie, startangle=90,
                                        textprops={'fontsize': 11, 'fontweight': 'bold'})
    ax3.set_title('Severity Concordance', fontsize=13, fontweight='bold')
    
    print(f"   Severity concordance: {concordance.get(True, 0)/len(df_both_copy)*100:.1f}% same severity level")
    
    # Average severity by presence pattern
    ax4 = axes[1, 1]
    
    patterns = []
    pattern_labels = []
    
    anxiety_only_obs = df_ad[(df_ad[anxiety_col] > 0) & (df_ad[depression_col] == 0)]
    if len(anxiety_only_obs) > 0:
        patterns.append(anxiety_only_obs[anxiety_col].mean())
        pattern_labels.append(f'Anxiety Only\n(n={len(anxiety_only_obs)})')
    
    depression_only_obs = df_ad[(df_ad[depression_col] > 0) & (df_ad[anxiety_col] == 0)]
    if len(depression_only_obs) > 0:
        patterns.append(depression_only_obs[depression_col].mean())
        pattern_labels.append(f'Depression Only\n(n={len(depression_only_obs)})')
    
    both_obs = df_ad[(df_ad[anxiety_col] > 0) & (df_ad[depression_col] > 0)]
    if len(both_obs) > 0:
        patterns.extend([both_obs[anxiety_col].mean(), both_obs[depression_col].mean()])
        pattern_labels.extend([f'Both: Anxiety\n(n={len(both_obs)})', 
                              f'Both: Depression\n(n={len(both_obs)})'])
    
    colors_bar = ['#FF6B6B', '#4ECDC4', '#FFB6B6', '#9CE8E0'][:len(patterns)]
    ax4.bar(range(len(patterns)), patterns, color=colors_bar, edgecolor='black', linewidth=1.5)
    ax4.set_ylabel('Average Severity', fontsize=12, fontweight='bold')
    ax4.set_title('Severity by Presence Pattern', fontsize=13, fontweight='bold')
    ax4.set_xticks(range(len(patterns)))
    ax4.set_xticklabels(pattern_labels, fontsize=9)
    ax4.grid(axis='y', alpha=0.3)
    
    for i, v in enumerate(patterns):
        ax4.text(i, v + 0.05, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "06_anxiety_depression_relationship.png"), 
               dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 06_anxiety_depression_relationship.png")
    
    # ========================================================================
    # REPORT 7: USER-LEVEL PATTERNS & PERSISTENCE
    # ========================================================================
    print("\n" + "-" * 70)
    print("REPORT 7: User-Level Patterns & Symptom Persistence")
    print("-" * 70)
    
    # Calculate user-level statistics
    user_stats = df_ad.groupby(user_col).agg({
        anxiety_col: ['mean', 'max', 'std', lambda x: (x > 0).sum()],
        depression_col: ['mean', 'max', 'std', lambda x: (x > 0).sum()],
        date_col: 'count'
    })
    
    user_stats.columns = ['anxiety_mean', 'anxiety_max', 'anxiety_std', 'anxiety_days',
                         'depression_mean', 'depression_max', 'depression_std', 'depression_days',
                         'total_days']
    
    user_stats['anxiety_persistence'] = user_stats['anxiety_days'] / user_stats['total_days'] * 100
    user_stats['depression_persistence'] = user_stats['depression_days'] / user_stats['total_days'] * 100
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Persistence distribution
    ax1 = axes[0, 0]
    ax1.hist(user_stats[user_stats['anxiety_persistence'] > 0]['anxiety_persistence'], 
            bins=20, alpha=0.6, label='Anxiety', color='#FF6B6B', edgecolor='black')
    ax1.hist(user_stats[user_stats['depression_persistence'] > 0]['depression_persistence'], 
            bins=20, alpha=0.6, label='Depression', color='#4ECDC4', edgecolor='black')
    ax1.set_xlabel('Persistence (% of Days)', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Number of Users', fontsize=11, fontweight='bold')
    ax1.set_title('Symptom Persistence Distribution', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    print(f"   Persistence metrics:")
    print(f"     Anxiety: Mean={user_stats['anxiety_persistence'].mean():.1f}%, "
          f"Median={user_stats['anxiety_persistence'].median():.1f}%")
    print(f"     Depression: Mean={user_stats['depression_persistence'].mean():.1f}%, "
          f"Median={user_stats['depression_persistence'].median():.1f}%")
    
    # Average severity by user
    ax2 = axes[0, 1]
    ax2.scatter(user_stats['anxiety_mean'], user_stats['depression_mean'], 
               alpha=0.5, s=50, c='#9B59B6')
    ax2.set_xlabel('Average Anxiety Severity', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Average Depression Severity', fontsize=11, fontweight='bold')
    ax2.set_title('User-Level Average Severity', fontsize=12, fontweight='bold')
    ax2.grid(alpha=0.3)
    
    # Variability (std) comparison
    ax3 = axes[0, 2]
    users_with_both = user_stats[(user_stats['anxiety_days'] > 0) & (user_stats['depression_days'] > 0)]
    ax3.scatter(users_with_both['anxiety_std'], users_with_both['depression_std'], 
               alpha=0.5, s=50, c='#E74C3C')
    ax3.set_xlabel('Anxiety Std Dev', fontsize=11, fontweight='bold')
    ax3.set_ylabel('Depression Std Dev', fontsize=11, fontweight='bold')
    ax3.set_title('Severity Variability by User', fontsize=12, fontweight='bold')
    ax3.grid(alpha=0.3)
    
    # User clustering by persistence pattern
    ax4 = axes[1, 0]
    
    def persistence_category(pct):
        if pct == 0:
            return 'Never'
        elif pct < 25:
            return 'Rare (<25%)'
        elif pct < 50:
            return 'Sometimes (25-50%)'
        elif pct < 75:
            return 'Often (50-75%)'
        else:
            return 'Frequent (>75%)'
    
    user_stats['anxiety_cat'] = user_stats['anxiety_persistence'].apply(persistence_category)
    user_stats['depression_cat'] = user_stats['depression_persistence'].apply(persistence_category)
    
    crosstab = pd.crosstab(user_stats['anxiety_cat'], user_stats['depression_cat'])
    category_order = ['Never', 'Rare (<25%)', 'Sometimes (25-50%)', 'Often (50-75%)', 'Frequent (>75%)']
    crosstab = crosstab.reindex(index=category_order, columns=category_order, fill_value=0)
    
    sns.heatmap(crosstab, annot=True, fmt='d', cmap='YlOrRd', ax=ax4, 
               cbar_kws={'label': 'Number of Users'})
    ax4.set_xlabel('Depression Persistence', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Anxiety Persistence', fontsize=11, fontweight='bold')
    ax4.set_title('User Clustering by Persistence Pattern', fontsize=12, fontweight='bold')
    
    # Days tracked distribution
    ax5 = axes[1, 1]
    anxiety_days_dist = user_stats[user_stats['anxiety_days'] > 0]['anxiety_days']
    depression_days_dist = user_stats[user_stats['depression_days'] > 0]['depression_days']
    
    ax5.hist(anxiety_days_dist, bins=30, alpha=0.6, label=f'Anxiety (mean={anxiety_days_dist.mean():.0f})', 
            color='#FF6B6B', edgecolor='black')
    ax5.hist(depression_days_dist, bins=30, alpha=0.6, label=f'Depression (mean={depression_days_dist.mean():.0f})', 
            color='#4ECDC4', edgecolor='black')
    ax5.set_xlabel('Number of Days Tracked', fontsize=11, fontweight='bold')
    ax5.set_ylabel('Number of Users', fontsize=11, fontweight='bold')
    ax5.set_title('Days Tracked per User', fontsize=12, fontweight='bold')
    ax5.legend()
    ax5.grid(axis='y', alpha=0.3)
    
    # Severity profile categories
    ax6 = axes[1, 2]
    
    # Get anxiety and depression severity values for median calculation
    anxiety_vals = df_ad[df_ad[anxiety_col] > 0][anxiety_col]
    depression_vals = df_ad[df_ad[depression_col] > 0][depression_col]
    
    def severity_profile(row):
        a_high = row['anxiety_mean'] > anxiety_vals.median()
        d_high = row['depression_mean'] > depression_vals.median()
        
        if row['anxiety_days'] == 0 and row['depression_days'] == 0:
            return 'Neither'
        elif row['anxiety_days'] > 0 and row['depression_days'] == 0:
            return 'Anxiety Only'
        elif row['depression_days'] > 0 and row['anxiety_days'] == 0:
            return 'Depression Only'
        elif a_high and d_high:
            return 'Both High'
        elif a_high and not d_high:
            return 'High Anxiety,\nLow Depression'
        elif d_high and not a_high:
            return 'Low Anxiety,\nHigh Depression'
        else:
            return 'Both Low'
    
    user_stats['profile'] = user_stats.apply(severity_profile, axis=1)
    profile_counts = user_stats['profile'].value_counts()
    
    colors_profile = {'Neither': '#F3F3F3', 'Anxiety Only': '#FF6B6B', 
                     'Depression Only': '#4ECDC4', 'Both High': '#E74C3C',
                     'High Anxiety,\nLow Depression': '#FFB6B6', 
                     'Low Anxiety,\nHigh Depression': '#9CE8E0',
                     'Both Low': '#95E1D3'}
    
    colors_list = [colors_profile.get(cat, '#CCCCCC') for cat in profile_counts.index]
    
    ax6.bar(range(len(profile_counts)), profile_counts.values, 
           color=colors_list, edgecolor='black', linewidth=1.5)
    ax6.set_ylabel('Number of Users', fontsize=11, fontweight='bold')
    ax6.set_title('User Severity Profiles', fontsize=12, fontweight='bold')
    ax6.set_xticks(range(len(profile_counts)))
    ax6.set_xticklabels(profile_counts.index, rotation=45, ha='right', fontsize=9)
    ax6.grid(axis='y', alpha=0.3)
    
    for i, v in enumerate(profile_counts.values):
        ax6.text(i, v + max(profile_counts.values)*0.02, str(v), 
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(ad_fig_path, "07_user_patterns_persistence.png"), 
               dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: 07_user_patterns_persistence.png")
    
    # ========================================================================
    # FINAL SUMMARY
    # ========================================================================
    print("\n" + "=" * 70)
    print("✅ ANXIETY & DEPRESSION ANALYSIS COMPLETE!")
    print("=" * 70)
    print(f"\n📁 Generated reports saved to: {ad_fig_path}")
    print(f"   01_prevalence.png")
    print(f"   02_demographics.png")
    print(f"   03_severity_temporal.png")
    print(f"   04_comorbidities.png")
    print(f"   05_correlations.png")
    print(f"   06_anxiety_depression_relationship.png")
    print(f"   07_user_patterns_persistence.png")
    
    print(f"\n📊 Key Findings:")
    print(f"   • {anxiety_users} users tracked anxiety ({anxiety_users/total_users*100:.1f}%)")
    print(f"   • {depression_users} users tracked depression ({depression_users/total_users*100:.1f}%)")
    print(f"   • {both_users} users tracked both ({both_users/total_users*100:.1f}%)")
    print(f"   • Anxiety-Depression correlation: r={corr_matrix.loc[anxiety_col, depression_col]:.3f}")
    print(f"   • Average persistence - Anxiety: {user_stats['anxiety_persistence'].mean():.1f}%, "
          f"Depression: {user_stats['depression_persistence'].mean():.1f}%")
    print("=" * 70)


----------------------------------------------------------------------
REPORT 4: Comorbidity Analysis
----------------------------------------------------------------------
   Top comorbidities with anxiety:
     Anxiety: 100.0%
     Fatigue: 62.7%
     Depression: 49.7%
     Headache: 36.6%
     Nausea: 28.3%
   Top comorbidities with depression:
     Depression: 100.0%
     Anxiety: 68.4%
     Fatigue: 67.1%
     Headache: 36.2%
     Cognitive Dysfunction: 29.9%
✓ Saved: 04_comorbidities.png

----------------------------------------------------------------------
REPORT 5: Correlation Analysis
----------------------------------------------------------------------
   Anxiety-Depression correlation: r=0.597
   Top correlates with anxiety:
     Condition Depression: r=0.597
     Gastroesophageal Reflux: r=0.452
     Sadness: r=0.418
     Hip Pain: r=0.389
     Fatigue: r=0.369
   Top correlates with depression:
     Sadness: r=0.798
     Condition Anxiety: r=0.597
     Fatigue: r=0.420


# Random Forest Forecasting: Data Preparation

## Current Dataset Status

**NOT ready for your forecasting task** - Your current dataset needs restructuring.

### The Problem

Your current dataset has **one row per user per day**:
- Row format: `[user_id, date, depression_value, anxiety_value, ...]`
- You have 72 users with depression data, 47 with 20+ days
- **Issue**: You can't use "first 10 days to predict next 10 days" with this structure

### What You Need

A **windowed dataset** where each row represents one prediction instance:

**Input Features (from days 1-10):**
- `depression_day1` through `depression_day10` (raw values)
- `depression_mean_10d`, `depression_std_10d`, `depression_min_10d`, `depression_max_10d`
- `anxiety_mean_10d`, `fatigue_mean_10d`, etc. (aggregated from all symptoms/conditions)
- `age`, `sex`, `country` (demographic context)
- `condition_fibromyalgia`, `condition_ibs`, etc. (binary: was it tracked in days 1-10?)

**Target Variables (days 11-20):**
- `target_depression_day11` through `target_depression_day20`

### Example Transformed Row

Instead of 20 rows (one per day), you'd have:
```
user_id=5, depression_day1=2.0, depression_day2=3.0, ..., depression_day10=2.5,
depression_mean_10d=2.3, anxiety_mean_10d=1.8, age=32, sex=female, country=US,
condition_fibromyalgia=1, condition_ibs=0, ...
target_depression_day11=2.0, target_depression_day12=1.5, ..., target_depression_day20=2.2
```

## Solution: Run the Next Cell

The next cell will create this properly formatted dataset for Random Forest training.

In [13]:
# ============================================================================
# CREATE RANDOM FOREST READY DATASET
# Transform from daily observations to windowed prediction format
# ============================================================================

print("="*70)
print("CREATING RANDOM FOREST FORECASTING DATASET")
print("="*70)

import pandas as pd
import numpy as np
from datetime import timedelta

# Configuration
WINDOW_SIZE = 10  # Days to use for features (input)
FORECAST_HORIZON = 10  # Days to predict (output)
TOTAL_DAYS_NEEDED = WINDOW_SIZE + FORECAST_HORIZON  # 20 days

# Load cleaned data
df = pd.read_csv(f"{CONFIG['output_path']}/flaredown_cleaned.csv")
df['checkin_date'] = pd.to_datetime(df['checkin_date'])

print(f"\n📊 Source Dataset:")
print(f"   - Total rows: {len(df):,}")
print(f"   - Total users: {df['user_seq_id'].nunique()}")
print(f"   - Users with depression: {df[df['condition_depression'].notna()]['user_seq_id'].nunique()}")

# Get base feature columns (not lag/rolling features)
symptom_cols = [col for col in df.columns if col.startswith('symptom_') and 'lag' not in col and 'rolling' not in col]
condition_cols = [col for col in df.columns if col.startswith('condition_') and 'lag' not in col and 'rolling' not in col]
demographic_cols = ['age', 'sex', 'country']

print(f"\n📋 Feature Categories:")
print(f"   - Symptoms: {len(symptom_cols)}")
print(f"   - Conditions: {len(condition_cols)}")
print(f"   - Demographics: {len([c for c in demographic_cols if c in df.columns])}")

# ============================================================================
# STEP 1: Find users with sufficient depression data
# ============================================================================

print(f"\n🔍 Finding users with {TOTAL_DAYS_NEEDED}+ consecutive depression observations...")

depression_data = df[df['condition_depression'].notna()].copy()
depression_data = depression_data.sort_values(['user_seq_id', 'checkin_date'])

valid_user_windows = []

for user_id in depression_data['user_seq_id'].unique():
    user_df = depression_data[depression_data['user_seq_id'] == user_id].copy()
    user_df = user_df.sort_values('checkin_date').reset_index(drop=True)
    
    # Find all possible 20-day windows
    for start_idx in range(len(user_df) - TOTAL_DAYS_NEEDED + 1):
        end_idx = start_idx + TOTAL_DAYS_NEEDED
        window = user_df.iloc[start_idx:end_idx]
        
        # Check if dates are reasonably consecutive (allow up to 2-day gaps)
        date_diffs = window['checkin_date'].diff().dt.days
        max_gap = date_diffs[1:].max()
        
        if pd.notna(max_gap) and max_gap <= 2:
            # Valid window found
            valid_user_windows.append({
                'user_seq_id': user_id,
                'window_start_idx': start_idx,
                'window_end_idx': end_idx,
                'start_date': window['checkin_date'].iloc[0],
                'end_date': window['checkin_date'].iloc[-1]
            })

print(f"   ✓ Found {len(valid_user_windows)} valid 20-day windows")
print(f"   ✓ From {len(set([w['user_seq_id'] for w in valid_user_windows]))} unique users")

if len(valid_user_windows) == 0:
    print("\n❌ ERROR: No valid windows found!")
    print("   Reduce WINDOW_SIZE or FORECAST_HORIZON, or check your data quality")
else:
    # ========================================================================
    # STEP 2: Create windowed dataset
    # ========================================================================
    
    print(f"\n🔨 Creating windowed features...")
    
    rf_dataset = []
    
    for window_info in valid_user_windows:
        user_id = window_info['user_seq_id']
        start_idx = window_info['window_start_idx']
        end_idx = window_info['window_end_idx']
        
        # Get this user's full data
        user_df = depression_data[depression_data['user_seq_id'] == user_id].sort_values('checkin_date').reset_index(drop=True)
        window = user_df.iloc[start_idx:end_idx]
        
        # Split into input (days 1-10) and target (days 11-20)
        input_window = window.iloc[:WINDOW_SIZE]
        target_window = window.iloc[WINDOW_SIZE:WINDOW_SIZE + FORECAST_HORIZON]
        
        # Initialize row
        row = {
            'user_seq_id': user_id,
            'window_start_date': window['checkin_date'].iloc[0],
            'window_end_date': window['checkin_date'].iloc[-1]
        }
        
        # Add demographics (from first day of window)
        for demo_col in demographic_cols:
            if demo_col in input_window.columns:
                row[demo_col] = input_window[demo_col].iloc[0]
        
        # Add depression daily values (days 1-10)
        for day in range(WINDOW_SIZE):
            if day < len(input_window):
                row[f'depression_day{day+1}'] = input_window['condition_depression'].iloc[day]
            else:
                row[f'depression_day{day+1}'] = np.nan
        
        # Add depression aggregates (days 1-10)
        depression_values = input_window['condition_depression'].dropna()
        if len(depression_values) > 0:
            row['depression_mean_10d'] = depression_values.mean()
            row['depression_std_10d'] = depression_values.std() if len(depression_values) > 1 else 0
            row['depression_min_10d'] = depression_values.min()
            row['depression_max_10d'] = depression_values.max()
            row['depression_median_10d'] = depression_values.median()
        else:
            row['depression_mean_10d'] = np.nan
            row['depression_std_10d'] = np.nan
            row['depression_min_10d'] = np.nan
            row['depression_max_10d'] = np.nan
            row['depression_median_10d'] = np.nan
        
        # Add anxiety aggregates (days 1-10) if exists
        if 'condition_anxiety' in input_window.columns:
            anxiety_values = input_window['condition_anxiety'].dropna()
            if len(anxiety_values) > 0:
                row['anxiety_mean_10d'] = anxiety_values.mean()
                row['anxiety_std_10d'] = anxiety_values.std() if len(anxiety_values) > 1 else 0
        
        # Add mean values for all symptoms (days 1-10)
        for symptom_col in symptom_cols:
            if symptom_col in input_window.columns:
                symptom_values = input_window[symptom_col].dropna()
                if len(symptom_values) > 0:
                    clean_name = symptom_col.replace('symptom_', '')
                    row[f'{clean_name}_mean_10d'] = symptom_values.mean()
        
        # Add binary indicators for conditions (was it present in days 1-10?)
        for condition_col in condition_cols:
            if condition_col in input_window.columns:
                clean_name = condition_col.replace('condition_', '')
                # 1 if condition was tracked at least once in the window, 0 otherwise
                has_condition = (input_window[condition_col] > 0).any()
                row[f'has_{clean_name}'] = 1 if has_condition else 0
        
        # Add temporal features
        row['window_day_of_week'] = input_window['checkin_date'].iloc[0].dayofweek
        row['window_month'] = input_window['checkin_date'].iloc[0].month
        row['window_is_weekend'] = 1 if input_window['checkin_date'].iloc[0].dayofweek >= 5 else 0
        
        # Add TARGET variables (days 11-20 of depression)
        for day in range(FORECAST_HORIZON):
            if day < len(target_window):
                row[f'target_depression_day{day+11}'] = target_window['condition_depression'].iloc[day]
            else:
                row[f'target_depression_day{day+11}'] = np.nan
        
        rf_dataset.append(row)
    
    # Convert to DataFrame
    df_rf = pd.DataFrame(rf_dataset)
    
    print(f"   ✓ Created {len(df_rf)} training instances")
    print(f"   ✓ Feature columns: {len([c for c in df_rf.columns if not c.startswith('target_')])}")
    print(f"   ✓ Target columns: {len([c for c in df_rf.columns if c.startswith('target_')])}")
    
    # ========================================================================
    # STEP 3: Data Quality Check
    # ========================================================================
    
    print(f"\n📊 Dataset Quality Report:")
    
    # Check for complete cases (no missing targets)
    target_cols = [c for c in df_rf.columns if c.startswith('target_')]
    complete_cases = df_rf[target_cols].notna().all(axis=1).sum()
    print(f"   - Complete cases (no missing targets): {complete_cases} / {len(df_rf)}")
    print(f"   - Incomplete cases: {len(df_rf) - complete_cases}")
    
    # Feature completeness
    feature_cols = [c for c in df_rf.columns if not c.startswith('target_') and c not in ['user_seq_id', 'window_start_date', 'window_end_date']]
    missing_per_feature = df_rf[feature_cols].isna().sum()
    features_with_missing = (missing_per_feature > 0).sum()
    print(f"   - Features with missing values: {features_with_missing} / {len(feature_cols)}")
    
    # Target statistics
    for target_col in target_cols[:3]:  # Show first 3 targets
        values = df_rf[target_col].dropna()
        if len(values) > 0:
            print(f"   - {target_col}: mean={values.mean():.2f}, std={values.std():.2f}, range=[{values.min():.1f}, {values.max():.1f}]")
    
    # ========================================================================
    # STEP 4: Export
    # ========================================================================
    
    print(f"\n💾 Exporting Random Forest dataset...")
    
    # Save full dataset
    rf_output_path = f"{CONFIG['output_path']}/rf_depression_forecast.csv"
    df_rf.to_csv(rf_output_path, index=False)
    print(f"   ✓ Saved: {rf_output_path}")
    
    # Create train/test split (80/20)
    from sklearn.model_selection import train_test_split
    
    # Split by users to avoid data leakage
    unique_users = df_rf['user_seq_id'].unique()
    train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)
    
    df_rf_train = df_rf[df_rf['user_seq_id'].isin(train_users)]
    df_rf_test = df_rf[df_rf['user_seq_id'].isin(test_users)]
    
    df_rf_train.to_csv(f"{CONFIG['output_path']}/rf_depression_forecast_train.csv", index=False)
    df_rf_test.to_csv(f"{CONFIG['output_path']}/rf_depression_forecast_test.csv", index=False)
    
    print(f"   ✓ Train set: {len(df_rf_train)} instances ({len(train_users)} users)")
    print(f"   ✓ Test set: {len(df_rf_test)} instances ({len(test_users)} users)")
    
    # ========================================================================
    # STEP 5: Usage Instructions
    # ========================================================================
    
    print("\n" + "="*70)
    print("✅ RANDOM FOREST DATASET READY!")
    print("="*70)
    
    print("\n📖 How to use this dataset:")
    print("\n1. LOAD THE DATA:")
    print("   df_train = pd.read_csv('cleaned_data/rf_depression_forecast_train.csv')")
    print("   df_test = pd.read_csv('cleaned_data/rf_depression_forecast_test.csv')")
    
    print("\n2. PREPARE FEATURES AND TARGETS:")
    print("   # Feature columns (everything except targets and IDs)")
    print("   feature_cols = [c for c in df_train.columns ")
    print("                   if not c.startswith('target_') ")
    print("                   and c not in ['user_seq_id', 'window_start_date', 'window_end_date']]")
    print("   ")
    print("   # Target columns (days 11-20 depression)")
    print("   target_cols = [c for c in df_train.columns if c.startswith('target_')]")
    print("   ")
    print("   X_train = df_train[feature_cols]")
    print("   y_train = df_train[target_cols]")
    print("   X_test = df_test[feature_cols]")
    print("   y_test = df_test[target_cols]")
    
    print("\n3. HANDLE MISSING VALUES:")
    print("   from sklearn.impute import SimpleImputer")
    print("   imputer = SimpleImputer(strategy='mean')")
    print("   X_train = imputer.fit_transform(X_train)")
    print("   X_test = imputer.transform(X_test)")
    
    print("\n4. TRAIN RANDOM FOREST:")
    print("   from sklearn.ensemble import RandomForestRegressor")
    print("   from sklearn.multioutput import MultiOutputRegressor")
    print("   ")
    print("   # Use MultiOutputRegressor for multiple targets (10 days)")
    print("   rf = MultiOutputRegressor(RandomForestRegressor(")
    print("       n_estimators=100,")
    print("       max_depth=10,")
    print("       min_samples_split=5,")
    print("       random_state=42,")
    print("       n_jobs=-1")
    print("   ))")
    print("   ")
    print("   rf.fit(X_train, y_train)")
    print("   predictions = rf.predict(X_test)")
    
    print("\n5. EVALUATE:")
    print("   from sklearn.metrics import mean_squared_error, mean_absolute_error")
    print("   ")
    print("   for i, target_col in enumerate(target_cols):")
    print("       y_true = y_test.iloc[:, i]")
    print("       y_pred = predictions[:, i]")
    print("       mse = mean_squared_error(y_true, y_pred)")
    print("       mae = mean_absolute_error(y_true, y_pred)")
    print(f"       print(f'Day {{i+11}}: MSE={{mse:.3f}}, MAE={{mae:.3f}}')")
    
    print("\n" + "="*70)
    
    # Display sample
    print("\n📋 Sample of transformed data:")
    sample_cols = ['user_seq_id', 'depression_day1', 'depression_day2', 'depression_mean_10d', 
                   'age', 'sex', 'target_depression_day11', 'target_depression_day12']
    available_cols = [c for c in sample_cols if c in df_rf.columns]
    print(df_rf[available_cols].head(3).to_string())

CREATING RANDOM FOREST FORECASTING DATASET

📊 Source Dataset:
   - Total rows: 198,063
   - Total users: 2221

📊 Source Dataset:
   - Total rows: 198,063
   - Total users: 2221
   - Users with depression: 800

📋 Feature Categories:
   - Symptoms: 84
   - Conditions: 60
   - Demographics: 3

🔍 Finding users with 20+ consecutive depression observations...
   - Users with depression: 800

📋 Feature Categories:
   - Symptoms: 84
   - Conditions: 60
   - Demographics: 3

🔍 Finding users with 20+ consecutive depression observations...
   ✓ Found 26696 valid 20-day windows
   ✓ From 489 unique users

🔨 Creating windowed features...
   ✓ Found 26696 valid 20-day windows
   ✓ From 489 unique users

🔨 Creating windowed features...
   ✓ Created 26696 training instances
   ✓ Feature columns: 170
   ✓ Target columns: 10

📊 Dataset Quality Report:
   - Complete cases (no missing targets): 26696 / 26696
   - Incomplete cases: 0
   - Features with missing values: 89 / 167
   - target_depression_day11:

# Summary: Dataset Transformation Complete

## What Changed?

### BEFORE (Current Dataset)
**Structure:** One row per user per day
```
user_id | date       | depression | anxiety | fatigue | age | sex
1       | 2018-03-01 | 2.0        | 1.0     | 3.0     | 32  | F
1       | 2018-03-02 | 2.5        | 1.5     | 3.5     | 32  | F
1       | 2018-03-03 | 2.0        | 2.0     | 3.0     | 32  | F
...
```

**Problem:** Cannot directly use "first 10 days to predict next 10 days"

### AFTER (Random Forest Ready)
**Structure:** One row per prediction window per user
```
user | depression_day1 | ... | depression_day10 | depression_mean_10d | anxiety_mean_10d | age | sex | target_day11 | ... | target_day20
1    | 2.0            | ... | 2.5             | 2.3                | 1.5             | 32  | F   | 2.0         | ... | 2.2
1    | 2.5            | ... | 3.0             | 2.8                | 1.8             | 32  | F   | 1.8         | ... | 2.5
...
```

**Solution:** Each row contains complete 10-day window as features + 10-day forecast as targets

## Key Features Created

**Input Features (from days 1-10):**
- Raw daily depression values: `depression_day1` through `depression_day10`
- Depression statistics: `depression_mean_10d`, `depression_std_10d`, `depression_min_10d`, `depression_max_10d`
- All other symptom means: `fatigue_mean_10d`, `headache_mean_10d`, etc.
- Condition indicators: `has_anxiety`, `has_fibromyalgia`, `has_ibs`, etc. (binary)
- Demographics: `age`, `sex`, `country`
- Temporal features: `window_day_of_week`, `window_month`, `window_is_weekend`

**Target Variables (days 11-20):**
- `target_depression_day11` through `target_depression_day20`

## Files Generated

1. **`rf_depression_forecast.csv`** - Full dataset with all windows
2. **`rf_depression_forecast_train.csv`** - Training set (80% of users)
3. **`rf_depression_forecast_test.csv`** - Test set (20% of users)

## Next Steps

1. **Run the previous cell** to generate the Random Forest dataset
2. **Train your model** using the code examples provided
3. **Evaluate** on test set to measure forecasting accuracy
4. **Iterate** on features/model parameters to improve performance

# Pipeline Execution Summary

This cell provides a complete audit trail of what happened to your data from raw input to final outputs.

In [14]:
# ============================================================================
# PIPELINE EXECUTION SUMMARY
# ============================================================================

import pandas as pd
import os
from datetime import datetime

print("=" * 80)
print("PIPELINE EXECUTION SUMMARY")
print("=" * 80)
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

# ============================================================================
# 1. INPUT DATA (RAW)
# ============================================================================

print("\n[1] INPUT DATA (RAW)")
print("-" * 80)

try:
    raw_file = f"{CONFIG['raw_data_path']}/export.csv"
    if os.path.exists(raw_file):
        try:
            df_raw = pd.read_csv(raw_file)
        except:
            df_raw = pd.read_csv(raw_file, delimiter=';')
        
        raw_user_col = next((col for col in df_raw.columns if 'user' in col.lower()), None)
        raw_users = df_raw[raw_user_col].nunique() if raw_user_col else 'N/A'
        
        print(f"Source: {raw_file}")
        print(f"Rows: {len(df_raw):,}")
        print(f"Columns: {len(df_raw.columns)}")
        print(f"Users: {raw_users:,}" if isinstance(raw_users, int) else f"Users: {raw_users}")
        print(f"Size: {os.path.getsize(raw_file) / (1024*1024):.2f} MB")
        
        # Store for comparison
        raw_rows = len(df_raw)
        raw_cols = len(df_raw.columns)
        raw_users_count = raw_users if isinstance(raw_users, int) else 0
        
        del df_raw
    else:
        print(f"⚠️  Could not find {raw_file}")
        raw_rows, raw_cols, raw_users_count = 0, 0, 0
except Exception as e:
    print(f"⚠️  Error loading raw data: {e}")
    raw_rows, raw_cols, raw_users_count = 0, 0, 0

# ============================================================================
# 2. OUTPUT DATA STATISTICS
# ============================================================================

print("\n[2] OUTPUT DATA STATISTICS")
print("-" * 80)

cleaned_file = f"{CONFIG['output_path']}/flaredown_cleaned.csv"
if os.path.exists(cleaned_file):
    df_clean = pd.read_csv(cleaned_file)
    
    # Basic stats
    print(f"Output: {cleaned_file}")
    print(f"Rows: {len(df_clean):,}")
    print(f"Columns: {len(df_clean.columns):,}")
    print(f"Users: {df_clean['user_seq_id'].nunique():,}")
    print(f"Date Range: {df_clean['checkin_date'].min()} to {df_clean['checkin_date'].max()}")
    print(f"Size: {os.path.getsize(cleaned_file) / (1024*1024):.2f} MB")
    
    # Feature breakdown
    symptom_base = [c for c in df_clean.columns if c.startswith('symptom_') and 'lag' not in c and 'rolling' not in c]
    condition_base = [c for c in df_clean.columns if c.startswith('condition_') and 'lag' not in c and 'rolling' not in c]
    lag_features = [c for c in df_clean.columns if 'lag' in c]
    rolling_features = [c for c in df_clean.columns if 'rolling' in c]
    
    print(f"\nFeatures: {len(symptom_base)} symptoms, {len(condition_base)} conditions, {len(lag_features)} lags, {len(rolling_features)} rolling")
    
    # Data quality
    obs_per_user = df_clean.groupby('user_seq_id').size()
    print(f"Obs/User: mean={obs_per_user.mean():.1f}, median={obs_per_user.median():.1f}, min={obs_per_user.min()}, max={obs_per_user.max()}")
    
    # Depression stats
    if 'condition_depression' in df_clean.columns:
        depression_data = df_clean[df_clean['condition_depression'].notna()]
        if len(depression_data) > 0:
            print(f"Depression: {len(depression_data):,} rows, {depression_data['user_seq_id'].nunique()} users, mean={depression_data['condition_depression'].mean():.2f}")
    
    # Store for comparison
    final_rows = len(df_clean)
    final_cols = len(df_clean.columns)
    final_users = df_clean['user_seq_id'].nunique()
    
    del df_clean
else:
    print(f"⚠️  Could not find {cleaned_file}")
    final_rows, final_cols, final_users = 0, 0, 0

# ============================================================================
# 3. DATA TRANSFORMATION SUMMARY
# ============================================================================

print("\n[3] DATA TRANSFORMATION SUMMARY")
print("-" * 80)

if raw_rows > 0 and final_rows > 0:
    # Rows
    row_change = final_rows - raw_rows
    row_pct = (row_change / raw_rows) * 100
    print(f"Rows: {raw_rows:,} → {final_rows:,} ({row_change:+,} rows, {row_pct:+.1f}%)")
    
    # Columns
    col_change = final_cols - raw_cols
    print(f"Columns: {raw_cols} → {final_cols:,} ({col_change:+,} columns)")
    
    # Users
    if raw_users_count > 0:
        user_change = final_users - raw_users_count
        user_retention = (final_users / raw_users_count) * 100
        print(f"Users: {raw_users_count:,} → {final_users:,} ({user_change:+,} users, {user_retention:.1f}% retained)")
else:
    print("⚠️  Could not compute transformation statistics")

print("\n" + "=" * 80)
print(f"✅ Pipeline Complete | Output: {CONFIG['output_path']}")
print("=" * 80)

PIPELINE EXECUTION SUMMARY
Timestamp: 2025-10-24 09:54:26

[1] INPUT DATA (RAW)
--------------------------------------------------------------------------------
Source: raw_data//export.csv
Rows: 7,976,223
Columns: 9
Users: 42,283
Size: 654.39 MB

[2] OUTPUT DATA STATISTICS
--------------------------------------------------------------------------------
Source: raw_data//export.csv
Rows: 7,976,223
Columns: 9
Users: 42,283
Size: 654.39 MB

[2] OUTPUT DATA STATISTICS
--------------------------------------------------------------------------------
Output: cleaned_data//flaredown_cleaned.csv
Rows: 198,063
Columns: 1,450
Users: 2,221
Date Range: 2013-05-06 to 2019-12-06
Size: 367.68 MB

Features: 84 symptoms, 60 conditions, 1008 lags, 288 rolling
Obs/User: mean=89.2, median=54.0, min=28, max=1612
Output: cleaned_data//flaredown_cleaned.csv
Rows: 198,063
Columns: 1,450
Users: 2,221
Date Range: 2013-05-06 to 2019-12-06
Size: 367.68 MB

Features: 84 symptoms, 60 conditions, 1008 lags, 288 roll

In [15]:
# ============================================================================
# CREATE RANDOM FOREST READY DATASET FOR ANXIETY
# Transform from daily observations to windowed prediction format
# ============================================================================

print("="*70)
print("CREATING RANDOM FOREST FORECASTING DATASET FOR ANXIETY")
print("="*70)

import pandas as pd
import numpy as np
from datetime import timedelta

# Configuration
WINDOW_SIZE = 10  # Days to use for features (input)
FORECAST_HORIZON = 10  # Days to predict (output)
TOTAL_DAYS_NEEDED = WINDOW_SIZE + FORECAST_HORIZON  # 20 days

# Load cleaned data
df = pd.read_csv(f"{CONFIG['output_path']}/flaredown_cleaned.csv")
df['checkin_date'] = pd.to_datetime(df['checkin_date'])

print(f"\n📊 Source Dataset:")
print(f"   - Total rows: {len(df):,}")
print(f"   - Total users: {df['user_seq_id'].nunique()}")
print(f"   - Users with anxiety: {df[df['condition_anxiety'].notna()]['user_seq_id'].nunique()}")

# Get base feature columns (not lag/rolling features)
symptom_cols = [col for col in df.columns if col.startswith('symptom_') and 'lag' not in col and 'rolling' not in col]
condition_cols = [col for col in df.columns if col.startswith('condition_') and 'lag' not in col and 'rolling' not in col]
demographic_cols = ['age', 'sex', 'country']

print(f"\n📋 Feature Categories:")
print(f"   - Symptoms: {len(symptom_cols)}")
print(f"   - Conditions: {len(condition_cols)}")
print(f"   - Demographics: {len([c for c in demographic_cols if c in df.columns])}")

# ============================================================================
# STEP 1: Find users with sufficient anxiety data
# ============================================================================

print(f"\n🔍 Finding users with {TOTAL_DAYS_NEEDED}+ consecutive anxiety observations...")

anxiety_data = df[df['condition_anxiety'].notna()].copy()
anxiety_data = anxiety_data.sort_values(['user_seq_id', 'checkin_date'])

valid_user_windows = []

for user_id in anxiety_data['user_seq_id'].unique():
    user_df = anxiety_data[anxiety_data['user_seq_id'] == user_id].copy()
    user_df = user_df.sort_values('checkin_date').reset_index(drop=True)
    
    # Find all possible 20-day windows
    for start_idx in range(len(user_df) - TOTAL_DAYS_NEEDED + 1):
        end_idx = start_idx + TOTAL_DAYS_NEEDED
        window = user_df.iloc[start_idx:end_idx]
        
        # Check if dates are reasonably consecutive (allow up to 2-day gaps)
        date_diffs = window['checkin_date'].diff().dt.days
        max_gap = date_diffs[1:].max()
        
        if pd.notna(max_gap) and max_gap <= 2:
            # Valid window found
            valid_user_windows.append({
                'user_seq_id': user_id,
                'window_start_idx': start_idx,
                'window_end_idx': end_idx,
                'start_date': window['checkin_date'].iloc[0],
                'end_date': window['checkin_date'].iloc[-1]
            })

print(f"   ✓ Found {len(valid_user_windows)} valid 20-day windows")
print(f"   ✓ From {len(set([w['user_seq_id'] for w in valid_user_windows]))} unique users")

if len(valid_user_windows) == 0:
    print("\n❌ ERROR: No valid windows found!")
    print("   Reduce WINDOW_SIZE or FORECAST_HORIZON, or check your data quality")
else:
    # ========================================================================
    # STEP 2: Create windowed dataset
    # ========================================================================
    
    print(f"\n🔨 Creating windowed features...")
    
    rf_dataset = []
    
    for window_info in valid_user_windows:
        user_id = window_info['user_seq_id']
        start_idx = window_info['window_start_idx']
        end_idx = window_info['window_end_idx']
        
        # Get this user's full data
        user_df = anxiety_data[anxiety_data['user_seq_id'] == user_id].sort_values('checkin_date').reset_index(drop=True)
        window = user_df.iloc[start_idx:end_idx]
        
        # Split into input (days 1-10) and target (days 11-20)
        input_window = window.iloc[:WINDOW_SIZE]
        target_window = window.iloc[WINDOW_SIZE:WINDOW_SIZE + FORECAST_HORIZON]
        
        # Initialize row
        row = {
            'user_seq_id': user_id,
            'window_start_date': window['checkin_date'].iloc[0],
            'window_end_date': window['checkin_date'].iloc[-1]
        }
        
        # Add demographics (from first day of window)
        for demo_col in demographic_cols:
            if demo_col in input_window.columns:
                row[demo_col] = input_window[demo_col].iloc[0]
        
        # Add anxiety daily values (days 1-10)
        for day in range(WINDOW_SIZE):
            if day < len(input_window):
                row[f'anxiety_day{day+1}'] = input_window['condition_anxiety'].iloc[day]
            else:
                row[f'anxiety_day{day+1}'] = np.nan
        
        # Add anxiety aggregates (days 1-10)
        anxiety_values = input_window['condition_anxiety'].dropna()
        if len(anxiety_values) > 0:
            row['anxiety_mean_10d'] = anxiety_values.mean()
            row['anxiety_std_10d'] = anxiety_values.std() if len(anxiety_values) > 1 else 0
            row['anxiety_min_10d'] = anxiety_values.min()
            row['anxiety_max_10d'] = anxiety_values.max()
            row['anxiety_median_10d'] = anxiety_values.median()
        else:
            row['anxiety_mean_10d'] = np.nan
            row['anxiety_std_10d'] = np.nan
            row['anxiety_min_10d'] = np.nan
            row['anxiety_max_10d'] = np.nan
            row['anxiety_median_10d'] = np.nan
        
        # Add depression aggregates (days 1-10) if exists
        if 'condition_depression' in input_window.columns:
            depression_values = input_window['condition_depression'].dropna()
            if len(depression_values) > 0:
                row['depression_mean_10d'] = depression_values.mean()
                row['depression_std_10d'] = depression_values.std() if len(depression_values) > 1 else 0
        
        # Add mean values for all symptoms (days 1-10)
        for symptom_col in symptom_cols:
            if symptom_col in input_window.columns:
                symptom_values = input_window[symptom_col].dropna()
                if len(symptom_values) > 0:
                    clean_name = symptom_col.replace('symptom_', '')
                    row[f'{clean_name}_mean_10d'] = symptom_values.mean()
        
        # Add binary indicators for conditions (was it present in days 1-10?)
        for condition_col in condition_cols:
            if condition_col in input_window.columns:
                clean_name = condition_col.replace('condition_', '')
                # 1 if condition was tracked at least once in the window, 0 otherwise
                has_condition = (input_window[condition_col] > 0).any()
                row[f'has_{clean_name}'] = 1 if has_condition else 0
        
        # Add temporal features
        row['window_day_of_week'] = input_window['checkin_date'].iloc[0].dayofweek
        row['window_month'] = input_window['checkin_date'].iloc[0].month
        row['window_is_weekend'] = 1 if input_window['checkin_date'].iloc[0].dayofweek >= 5 else 0
        
        # Add TARGET variables (days 11-20 of anxiety)
        for day in range(FORECAST_HORIZON):
            if day < len(target_window):
                row[f'target_anxiety_day{day+11}'] = target_window['condition_anxiety'].iloc[day]
            else:
                row[f'target_anxiety_day{day+11}'] = np.nan
        
        rf_dataset.append(row)
    
    # Convert to DataFrame
    df_rf = pd.DataFrame(rf_dataset)
    
    print(f"   ✓ Created {len(df_rf)} training instances")
    print(f"   ✓ Feature columns: {len([c for c in df_rf.columns if not c.startswith('target_')])}")
    print(f"   ✓ Target columns: {len([c for c in df_rf.columns if c.startswith('target_')])}")
    
    # ========================================================================
    # STEP 3: Data Quality Check
    # ========================================================================
    
    print(f"\n📊 Dataset Quality Report:")
    
    # Check for complete cases (no missing targets)
    target_cols = [c for c in df_rf.columns if c.startswith('target_')]
    complete_cases = df_rf[target_cols].notna().all(axis=1).sum()
    print(f"   - Complete cases (no missing targets): {complete_cases} / {len(df_rf)}")
    print(f"   - Incomplete cases: {len(df_rf) - complete_cases}")
    
    # Feature completeness
    feature_cols = [c for c in df_rf.columns if not c.startswith('target_') and c not in ['user_seq_id', 'window_start_date', 'window_end_date']]
    missing_per_feature = df_rf[feature_cols].isna().sum()
    features_with_missing = (missing_per_feature > 0).sum()
    print(f"   - Features with missing values: {features_with_missing} / {len(feature_cols)}")
    
    # Target statistics
    for target_col in target_cols[:3]:  # Show first 3 targets
        values = df_rf[target_col].dropna()
        if len(values) > 0:
            print(f"   - {target_col}: mean={values.mean():.2f}, std={values.std():.2f}, range=[{values.min():.1f}, {values.max():.1f}]")
    
    # ========================================================================
    # STEP 4: Export
    # ========================================================================
    
    print(f"\n💾 Exporting Random Forest dataset for anxiety...")
    
    # Save full dataset
    rf_output_path = f"{CONFIG['output_path']}/rf_anxiety_forecast.csv"
    df_rf.to_csv(rf_output_path, index=False)
    print(f"   ✓ Saved: {rf_output_path}")
    
    # Create train/test split (80/20)
    from sklearn.model_selection import train_test_split
    
    # Split by users to avoid data leakage
    unique_users = df_rf['user_seq_id'].unique()
    train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)
    
    df_rf_train = df_rf[df_rf['user_seq_id'].isin(train_users)]
    df_rf_test = df_rf[df_rf['user_seq_id'].isin(test_users)]
    
    df_rf_train.to_csv(f"{CONFIG['output_path']}/rf_anxiety_forecast_train.csv", index=False)
    df_rf_test.to_csv(f"{CONFIG['output_path']}/rf_anxiety_forecast_test.csv", index=False)
    
    print(f"   ✓ Train set: {len(df_rf_train)} instances ({len(train_users)} users)")
    print(f"   ✓ Test set: {len(df_rf_test)} instances ({len(test_users)} users)")
    
    # ========================================================================
    # STEP 5: Usage Instructions
    # ========================================================================
    
    print("\n" + "="*70)
    print("✅ RANDOM FOREST ANXIETY DATASET READY!")
    print("="*70)
    
    print("\n📖 How to use this dataset:")
    print("\n1. LOAD THE DATA:")
    print("   df_train = pd.read_csv('cleaned_data/rf_anxiety_forecast_train.csv')")
    print("   df_test = pd.read_csv('cleaned_data/rf_anxiety_forecast_test.csv')")
    
    print("\n2. PREPARE FEATURES AND TARGETS:")
    print("   # Feature columns (everything except targets and IDs)")
    print("   feature_cols = [c for c in df_train.columns ")
    print("                   if not c.startswith('target_') ")
    print("                   and c not in ['user_seq_id', 'window_start_date', 'window_end_date']]")
    print("   ")
    print("   # Target columns (days 11-20 anxiety)")
    print("   target_cols = [c for c in df_train.columns if c.startswith('target_')]")
    print("   ")
    print("   X_train = df_train[feature_cols]")
    print("   y_train = df_train[target_cols]")
    print("   X_test = df_test[feature_cols]")
    print("   y_test = df_test[target_cols]")
    
    print("\n3. HANDLE MISSING VALUES:")
    print("   from sklearn.impute import SimpleImputer")
    print("   imputer = SimpleImputer(strategy='mean')")
    print("   X_train = imputer.fit_transform(X_train)")
    print("   X_test = imputer.transform(X_test)")
    
    print("\n4. TRAIN RANDOM FOREST:")
    print("   from sklearn.ensemble import RandomForestRegressor")
    print("   from sklearn.multioutput import MultiOutputRegressor")
    print("   ")
    print("   # Use MultiOutputRegressor for multiple targets (10 days)")
    print("   rf = MultiOutputRegressor(RandomForestRegressor(")
    print("       n_estimators=100,")
    print("       max_depth=10,")
    print("       min_samples_split=5,")
    print("       random_state=42,")
    print("       n_jobs=-1")
    print("   ))")
    print("   ")
    print("   rf.fit(X_train, y_train)")
    print("   predictions = rf.predict(X_test)")
    
    print("\n5. EVALUATE:")
    print("   from sklearn.metrics import mean_squared_error, mean_absolute_error")
    print("   ")
    print("   for i, target_col in enumerate(target_cols):")
    print("       y_true = y_test.iloc[:, i]")
    print("       y_pred = predictions[:, i]")
    print("       mse = mean_squared_error(y_true, y_pred)")
    print("       mae = mean_absolute_error(y_true, y_pred)")
    print(f"       print(f'Day {{i+11}}: MSE={{mse:.3f}}, MAE={{mae:.3f}}')")
    
    print("\n" + "="*70)
    
    # Display sample
    print("\n📋 Sample of transformed data:")
    sample_cols = ['user_seq_id', 'anxiety_day1', 'anxiety_day2', 'anxiety_mean_10d', 
                   'age', 'sex', 'target_anxiety_day11', 'target_anxiety_day12']
    available_cols = [c for c in sample_cols if c in df_rf.columns]
    print(df_rf[available_cols].head(3).to_string())

CREATING RANDOM FOREST FORECASTING DATASET FOR ANXIETY

📊 Source Dataset:
   - Total rows: 198,063
   - Total users: 2221
   - Users with anxiety: 937

📋 Feature Categories:
   - Symptoms: 84
   - Conditions: 60
   - Demographics: 3

🔍 Finding users with 20+ consecutive anxiety observations...
   ✓ Found 32488 valid 20-day windows
   ✓ From 581 unique users

🔨 Creating windowed features...
   ✓ Created 32488 training instances
   ✓ Feature columns: 170
   ✓ Target columns: 10

📊 Dataset Quality Report:
   - Complete cases (no missing targets): 32488 / 32488
   - Incomplete cases: 0
   - Features with missing values: 89 / 167
   - target_anxiety_day11: mean=1.47, std=1.15, range=[0.0, 4.0]
   - target_anxiety_day12: mean=1.47, std=1.15, range=[0.0, 4.0]
   - target_anxiety_day13: mean=1.47, std=1.15, range=[0.0, 4.0]

💾 Exporting Random Forest dataset for anxiety...
   ✓ Saved: cleaned_data//rf_anxiety_forecast.csv
   ✓ Train set: 25945 instances (464 users)
   ✓ Test set: 6543 instance